In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import numpy.matlib    
import pandas as pd
import scipy as sc
import csv
import math
import random
from sklearn.model_selection import train_test_split
from scipy.sparse import isspmatrix, csc_matrix, csr_matrix
from scipy.sparse.linalg import eigsh, svds
from scipy.linalg import svd
import os
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
import io

In [2]:
from skopt import gp_minimize
from skopt.space import Real, Integer

In [3]:
eps                  = np.finfo(float).eps

In [4]:
def createList(r1,r2):
    return [item for item in range(r1,r2 + 1)]

In [5]:
def userSim(Y): 
    print("In userSim")
    N,M              = Y.shape  
    
    Y01              = Y.copy()
    Y01[Y01>0]       = 1
    
    tmp              = (Y**2)@(Y01.T)
    tmp              = tmp + tmp.T
    tmp              = tmp - 2*(Y@(Y.T))
    
    simScore         = 1.0/(1 + np.sqrt(tmp))
    
    nominator        = Y01@Y01.T
    
    denominator      = np.sum(Y01,axis=1).reshape(-1,1)
    denominator      = np.matlib.repmat((denominator),1,N)
    denominator      = denominator + denominator.T
    denominator      = denominator - nominator
    
    B1               = nominator/denominator
    B1[np.isnan(B1)] = 0
    
    simScore         = simScore*B1
    return simScore

In [6]:
def evecs(A,nEvecs):
    npix             = A.shape[0]
    useSparse        = isspmatrix(A)
    
    dd               = 1/((np.sum(A,axis=0))+eps)
    dd               = np.sqrt(dd)

    if(useSparse):
        DD           = sc.sparse.diags(dd)
    else:
        DD           = np.diag(dd)
    
    L                = DD@A@DD
    
    if(useSparse==1):
        ss,v         = eigsh(L,nEvecs,sigma=1)
        ss           = np.flip(ss)
        V            = np.flip(v,axis=1)
    else:
        u,ss,v       = svd(L)
        V            = u[:, :nEvecs]
        
    ss               = ss.reshape(-1,1)
    ss               = ss[:nEvecs]
    
    return V,ss

In [7]:
def spectralClu(A=None,K=None): 
    # A: Affinity Matrix, Higher value -> more similar
    # K: Number of CLuster
    n                = A.shape[0]
    
    D                = np.sum(A,axis=1).reshape(-1,1)
    D[D==0]          = eps
    D                = sc.sparse.spdiags((1.0/np.sqrt(D)).T,0,n,n)
    
    L                = D@A@D
    V,evals          = evecs(L,K)
    
    Vnorm            = (np.linalg.norm(V,axis=1) + eps).reshape(-1,1)
    
    V                = V/Vnorm

    kmeans           = KMeans(n_clusters=K,max_iter=maxiter,random_state=0).fit(V)
    idx              = kmeans.predict(V)

    return idx

In [8]:
def aggregratePrediction(X,UGidx):
    n,m                  = X.shape
    K1                   = len(np.unique(UGidx))
    ctmp_d               = np.ones(n)
    ctmp                 = csc_matrix((ctmp_d,(UGidx.reshape(-1,),range(n))),shape=(K1,n))

    GFreq                = np.bincount(UGidx.reshape(-1,)).reshape(-1,1)
    XPerA                = (ctmp@X)/np.matlib.repmat((GFreq),1,m)
    
    
    return XPerA

In [9]:
def MAE(a,b):
    return np.sum(abs(a*(b!=0) - b))/np.sum(b!=0)

In [10]:
def RMSE(X,Y):
    tmp                        = ((Y - X)*(Y!=0))**2
    
    return math.sqrt(np.sum(tmp)/np.sum(Y!=0))

In [11]:
def AUC(Ypre,Yt,cutoff):
    userWithRating             = (np.sum(Yt!=0,axis=1)>0)
    Yt                         = Yt[userWithRating,:]
    Ypre                       = Ypre[userWithRating,:]
    N,M                        = Yt.shape
    auc                        = 0
    uWithPair                  = 0
    for user in range(N):
        userRating             = Yt[user,:]
        userPrediction         = Ypre[user, :]
        ratedIdx               = userRating>0
        binUserRating          = 2*(userRating[ratedIdx]>=cutoff)-1
        predForKnown           = userPrediction[ratedIdx]
        
        posIdx                 = np.where(binUserRating==1)
        posIdx                 = posIdx[0].T

        negIdx                 = np.where(binUserRating==-1)
        negIdx                 = negIdx[0]
        
        if(np.size(posIdx)!=0 and np.size(negIdx)!=0):
            negIdxGrid         = np.matlib.repmat(negIdx,len(posIdx),1)
            ttlNeg             = np.size(negIdxGrid)
            negIdxGrid         = negIdxGrid.reshape(ttlNeg,1,order='F')
            posIdxGrid         = np.matlib.repmat(posIdx.reshape(-1,1),len(negIdx),1)
            pairs              = np.concatenate((negIdxGrid,posIdxGrid),axis=1)
            pairsPred          = np.concatenate((pairs,predForKnown[negIdxGrid],predForKnown[posIdxGrid]),axis = 1)
            pairsPredWithScore = np.concatenate([pairsPred,(0.5*(pairsPred[:,2]==pairsPred[:,3]).reshape(-1,1)),(1*(pairsPred[:,2]<pairsPred[:,3]).reshape(-1,1))],axis=1)
            auc                = auc + (np.sum(pairsPredWithScore[:,4:6]))/pairsPredWithScore.shape[0]
            uWithPair          = uWithPair + 1    
    auc                        = auc/uWithPair
    
    return auc

In [12]:
def precAtK(Ypre,Yt,k,cutoff):
    # %Yt  : Test Set of size n *m
    # %Ypre: Prediction set of size n *m
    # %cutoff : threshold for relevant item
    # %k: precision@k

    userWithRating             = (np.sum(Yt!=0,axis=1)>0)
    Yt                         = Yt[userWithRating,:]
    Ypre                       = Ypre[userWithRating,:]
    N,M                        = Yt.shape
    prec                       = np.zeros((1,k))

    for user in range(N):
        userRating             = Yt[user, :]
        
        ratedRelevantIdx       = np.where(userRating>=cutoff)
        ratedRelevantIdx       = ratedRelevantIdx[0]

        ratedIdx               = np.where(userRating!=0)
        ratedIdx               = ratedIdx[0]

        userPrediction         = Ypre[user, :]
        prediction             = userPrediction[ratedIdx]
        sortesPIdxTmp          = np.argsort(prediction)[::-1]
        sortesPIdx             = ratedIdx[sortesPIdxTmp]
        nz                     = len(sortesPIdx)
        for kNo in range(k):
            intrsct            = np.intersect1d(sortesPIdx[0:(min(kNo,nz)+1)],ratedRelevantIdx)
            mx                 = max(min(kNo+1,nz),eps)
            prec[:,kNo]        = prec[:,kNo] + len(intrsct)/mx
            
    prec                       = prec/N
    
    return prec

In [13]:
def recallAtK(Ypre,Yt, k, cutoff):
    # %Yt  : Test Set of size n *m
    # %Ypre: Prediction set of size n *m
    # %cutoff : threshold for relevant item
    # %k: recall@k

    userWithRating             = (np.sum(Yt!=0,axis=1)>0)
    Yt                         = Yt[userWithRating,:]
    Ypre                       = Ypre[userWithRating,:]
    N,M                        = Yt.shape
    recall                     = np.zeros((1,k))
    
    for user in range(N):
        userRating             = Yt[user, :]
        userPrediction         = Ypre[user, :]
        
        ratedRelevantIdx       = np.where(userRating>=cutoff)
        ratedRelevantIdx       = ratedRelevantIdx[0]

        ratedIdx               = np.where(userRating!=0)
        ratedIdx               = ratedIdx[0]

        prediction             = userPrediction[ratedIdx]
        
        sortesPIdxTmp          = np.argsort(prediction)[::-1]
        sortesPIdx             = ratedIdx[sortesPIdxTmp]
        
        nz                     = len(sortesPIdx)
        for kNo in range(k):
            intrsct            = np.intersect1d(sortesPIdx[0:(min(kNo,nz)+1)],ratedRelevantIdx)
            mx                 = max(len(ratedRelevantIdx),eps)
            recall[:,kNo]      = recall[:,kNo] + len(intrsct)/mx
            
    recall                     = recall/N
    
    return recall 

In [14]:
def ndcgAtk(Ypre,Yt,k):
    # %Yt  : Test Set of size n *m
    # %Ypre: Prediction set of size n *m
    # %cutoff : threshold for relevant item
    # %k: precision@k

    res                        = np.zeros((1,k))
    cnt                        = 0
    N,M                        = Yt.shape

    for user in range(N):
        ratedIdx               = Yt[user,:]!=0

        userRating             = Yt[user,ratedIdx]
        userPrediction         = Ypre[user,ratedIdx]
        nz                     = len(userRating)

        ranks                  = np.zeros((1,nz))
        ideal_ranks            = np.zeros((1,nz))
        
        I                      = np.argsort(-userPrediction,axis=0)
        ideal_I                = np.argsort(-userRating,axis=0)

        ranks                  = I
        ideal_ranks            = ideal_I

        oriOrder               = np.argsort(ideal_ranks,axis=0)

        nominator              = userRating/(np.log(ranks+2))
        denominator            = userRating/(np.log(ideal_ranks + 2))
    
        nominator              = nominator[oriOrder]
        denominator            = denominator[oriOrder]
        
        if k > nz:
            nominator          = np.concatenate((nominator, np.zeros((k - nz))))
            denominator        = np.concatenate((denominator, np.zeros((k - nz))))
        elif k < nz:
            nominator          = nominator[0:k]
            denominator        = denominator[0:k]          

        if np.array(np.where(np.cumsum(denominator)== 0)).shape[1] != 0:
            tmp                = np.zeros((1,k))
        else:
            tmp                = np.cumsum(nominator) / np.cumsum(denominator)
            cnt                = cnt + 1
        
        res                    = res + tmp
        
    res                        = res/cnt
    
    return res

In [15]:
def EvaluationAllUpdated(Yprd,Y1,k,cutoff):
    mae                        = MAE(Yprd, Y1) 
    rmse                       = RMSE(Yprd, Y1)
    auc                        = AUC(Yprd, Y1, cutoff)
    precision                  = precAtK(Yprd,Y1, k, cutoff)
    recall                     = recallAtK(Yprd,Y1, k, cutoff)
    f1                         = (2*precision*recall)/(precision + recall + eps)
    ndcg                       = ndcgAtk(Yprd,Y1,k)

    return mae,rmse,auc,precision,recall,f1,ndcg

In [16]:
def EvaluationAllUpdated_N(Yprd,Y1,k,cutoff):
    mae                        = MAE(Yprd, Y1) 
    rmse                       = RMSE(Yprd, Y1)

    return mae,rmse

In [17]:
cols                     = ["Model","MAE","RMSE","AUC","Precision@1","Precision@5","Precision@10","Precision@20",
                            "Precision@30","Precision@40","Recall@1","Recall@5","Recall@10","Recall@20","Recall@30",
                            "Recall@40","F1@1","F1@5","F1@10","F1@20","F1@30","F1@40","NDCG@1","NDCG@5","NDCG@10",
                            "NDCG@20","NDCG@30","NDCG@40"]
Result_Trn               = pd.DataFrame(columns=cols)

In [18]:
cols                     = ["Model","MAE","RMSE"]
Result_Trn_CDR               = pd.DataFrame(columns=cols)
Result_Tst_CDR = pd.DataFrame(columns=cols)

In [19]:
def gradUnifiedLS(v,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx): 
    
    n,m                  = Y.shape
    U                    = v[0:n*d].reshape(n,d,order='F')
    V                    = v[n*d:n*d+m*d].reshape(m,d,order='F')
    UG                   = v[n*d+m*d:n*d+m*d+K1*d].reshape(K1,d,order='F')
    VG                   = v[n*d+m*d+K1*d:].reshape(K2,d,order='F')

    UGmatMU              = (UG[UGidx.reshape(-1,),:]) - U
    VGmatMV              = (VG[VGidx.reshape(-1,),:]) - V
    
    X                    = U@(V.T)
    Ygt0                 = Y>0
    YMXgt0               = (Y - X)*Ygt0

    regobj               = (lambda3/2)*(np.sum(U**2) + np.sum(V**2))

    lossobj              = 0
    lossobj              = lossobj + (0.5*(np.sum(YMXgt0**2)))
    lossobj              = lossobj + (lambda1/2)*(np.sum(UGmatMU**2) + np.sum(VGmatMV**2))
    
    dU                   = lambda3*U
    dV                   = lambda3*V
    
    dU                   = dU - YMXgt0@V
    dV                   = dV - (YMXgt0.T)@U

    dU                   = dU - (lambda1*UGmatMU)
    dV                   = dV - (lambda1*VGmatMV)

    ctmp_d               = np.ones(n)                   # convert UGidx to a K1-by-n matrix containing the k1 indicator vectors as row
    ctmp                 = csc_matrix((ctmp_d,(UGidx.reshape(-1,),range(n))),shape=(K1,n))
    dUG                  = lambda1*((ctmp)@UGmatMU)
    
    ctmp_d               = np.ones(m)                   # convert UGidx to a K2-by-m matrix containing the k2 indicator vectors as row
    ctmp                 = csc_matrix((ctmp_d,(VGidx.reshape(-1,),range(m))),shape=(K2,m))
    dVG                  = lambda1*((ctmp)@VGmatMV)
    
    obj                  = regobj + lossobj
   
    grad                 = np.concatenate((dU.flatten('F'),dV.flatten('F'),dUG.flatten('F'),dVG.flatten('F'))).reshape(-1,1)
    
    return obj,grad,lossobj,regobj

In [20]:
def gradUnifiedMMMF(v,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGIdx,VGIdx):

    n,m                  = Y.shape
    U                    = v[0:n*d].reshape(n,d,order='F')
    V                    = v[n*d:n*d+m*d].reshape(m,d,order='F')
    theta                = v[n*d+m*d:n*d+m*d+n*(l-1)].reshape(n,l-1,order='F')
    UG                   = v[n*d+m*d+n*(l-1):n*d+m*d+n*(l-1)+K1*d].reshape((K1,d), order='F')
    VG                   = v[n*d+m*d+n*(l-1)+K1*d:n*d+m*d+n*(l-1)+K1*d+K2*d].reshape((K2,d), order='F')
    thetaG               = v[n*d+m*d+n*(l-1)+K1*d+K2*d:].reshape((K1,l-1), order='F')
        
    UGmatMU              = (UG[UGIdx.reshape(-1,),:]) - U
    VGmatMV              = (VG[VGIdx.reshape(-1,),:]) - V
    thetaGMtheta         = (thetaG[UGIdx.reshape(-1,),:]) - theta
    
    X                    = U@V.T
    Ygt0                 = Y>0
    BX                   = X*Ygt0
    
    dU                   = lambda3*U
    dV                   = lambda3*V
    dtheta               = np.zeros((n, l-1))
    
    regobj               = (lambda3/2)*(np.sum(U**2) + np.sum(V**2))
    
    lossobj              = 0
    lossobj              = lossobj + (lambda1/2)*(np.sum(UGmatMU**2) + np.sum(VGmatMV**2) + np.sum(thetaGMtheta**2))
    
    for k in range(l-1):
        S                = Ygt0 - 2*(Y>k+1)
        BZ               = (theta[:,k].reshape(-1,1)@(np.ones([1,m])))*S - BX*S
        lossobj          = lossobj + sum(sum(h(BZ)))
        tmp              = hprime(BZ)*S
        dU               = dU - tmp@V
        dV               = dV - tmp.T@U
        dtheta[:,k]      = tmp@np.ones((m,))
        
    dU                   = dU - lambda1*UGmatMU
    dV                   = dV - lambda1*VGmatMV
    dtheta               = dtheta - lambda1*thetaGMtheta
    
    ctmp_d               = np.ones(n)                   # convert UGidx to a K1-by-n matrix containing the k1 indicator vectors as row
    ctmp                 = csc_matrix((ctmp_d,(UGIdx.reshape(-1,),range(n))),shape=(K1,n))
    dUG                  = np.multiply(lambda1,(ctmp@UGmatMU))
    dthetaG              = np.multiply(lambda1,(ctmp@thetaGMtheta))
 
    ctmp_d               = np.ones(m)                   # convert UGidx to a K2-by-m matrix containing the k2 indicator vectors as row
    ctmp                 = csc_matrix((ctmp_d,(VGIdx.reshape(-1,),range(m))),shape=(K2,m))
    dVG                  = np.multiply(lambda1,(ctmp@VGmatMV))
    
    obj                  = regobj + lossobj;            # obj is the objective function that we need to minimize

    grad                 = np.concatenate((dU.flatten('F'),dV.flatten('F'),dtheta.flatten('F'),dUG.flatten('F'),dVG.flatten('F'),dthetaG.flatten('F'))).reshape(-1,1)
    
    return obj,grad,lossobj,regobj

In [21]:
def conjgrad(x0,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx):
    J                     = []
    temp                  = 0
    nu                    = 0.1
    abstol                = 0                                 # stop if gradient magnitude goes below this
    allowNonDecrease      = 0                                 # don't stop if line search fails to find decrease
    digits                = 12                                # digits of precision to use for objective comparisons
    ogfun                 = objGrad
    ogcalls               = 0
    x                     = x0
    numiter               = 0
    j                     = 0
    alpha                 = 10**(-10)
    ogcalls               = ogcalls + 1
    obj,dx,lossobj,regobj = ogfun(x,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
    r                     = -dx
    s                     = r
    dirn                  = s
    deltanew              = (r.T)@dirn
    deltazero             = deltanew
    
    while ((numiter<maxiter) and (abs(deltanew)>tol*tol*abs(deltazero)) and (abs(deltanew)>abstol)):
        numiter           = numiter + 1
        j                 = j + 1
        print('\n %.4f' %obj)
        J.append(obj)
        prevobj           = obj

        if (alpha < 10**(-10)):
            alpha         = 10**(-10)

        alpha,obj,dx,ogc  = cgLineSearch(x,obj,dx,dirn,alpha,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,lossobj,regobj)
        ogcalls           = ogcalls + ogc
        
        temp              = temp + alpha
        x                 = x + alpha*dirn
        r                 = -dx
        deltaold          = deltanew
        deltamid          = r.T@s
        deltanew          = r.T@r
        beta              = (deltanew - deltamid) / deltaold
        dirn              = r + max(0, beta)*dirn
        if ((deltamid/deltanew >= nu) or (dirn.T@dx >= 0)):
            dirn          = r
            j             = 0
        s                 = r
        print(numiter)
        
    return x,numiter,ogcalls,J

In [22]:
def cgLineSearch(x0,obj0,dx0,direction,alpha,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,lobj,robj):
    seciter                   = 5                               # maximum number of quadratic interpolation iterations
    alpha0                    = alpha
    c1                        = 10**(-4)                        # required decrease in objective (relative to gradient)
    digits                    = 12                              # digits of precision to use for objective comparisons
    gamma                     = 10
    ogfun                     = objGrad
    
    if (alpha0 <= 0):                                           # check initial values
        print('alpha0 must be greater than zero')
    
    obj                       = obj0                            # begin line search
    dx                        = dx0
    etazero                   = (dx.T)@direction
    etaprev                   = etazero
    alpha                     = alpha0
    ogcalls                   = 0
    lossobj                   = lobj
    regobj                    = robj
    
    alpha,obj,dx,lossobj,regobj,ogcalls = findNonZeroAlpha(x0,alpha,direction,ogfun,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,obj,dx,lossobj,regobj)
    
    obj,dx,lossobj,regobj     = ogfun(x0+alpha*direction,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
    ogcalls                   = ogcalls + 1
    
    oldalpha,oldobj,olddx                           = saveAlpha(alpha,obj,dx)
    alpha,obj,dx,lossobj,regobj,ogcalls,doBacktrack = backtrack(x0,alpha,direction,ogfun,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,gamma,obj,dx,lossobj,regobj,digits,obj0,c1,etazero,oldalpha,oldobj,olddx)

    beta                      = alpha
    eta                       = dx.T@direction
    i                         = 0
    
    while((abs(eta)>c2*abs(etazero)) and (i<seciter) and (pround(obj,digits) <= pround(obj0,digits)) and (etaprev!=eta) and (np.sum((x0 + alpha*direction) != x0)>0)):
        beta                  = eta*beta / (etaprev - eta)
        oldalpha,oldobj,olddx = saveAlpha(alpha,obj,dx)
        alpha                 = alpha + beta
        if(alpha<=0):
            alpha             = 1
        etaprev               = eta
        i                     = i + 1
        obj,dx,lossobj,regobj = ogfun(x0+alpha*direction,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
        ogcalls               = ogcalls + 1
        eta                   = dx.T@direction

    alpha,obj,dx,lossobj,regobj,ogcalls             = findNonZeroAlpha(x0,alpha,direction,ogfun,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,obj,dx,lossobj,regobj)
    alpha,obj,dx,lossobj,regobj,ogcalls,doBacktrack = backtrack(x0,alpha,direction,ogfun,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,gamma,obj,dx,lossobj,regobj,digits,obj0,c1,etazero,oldalpha,oldobj,olddx)
    checkConditions(obj,digits,obj0,etazero,eta,x0,alpha,direction)
    
    return alpha,obj,dx,ogcalls 

In [23]:
def findNonZeroAlpha(x0,alpha,direction,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,obj,dx,lossobj,regobj):
    if(np.array_equal((x0+alpha*direction),x0)):                # Make sure alpha isn't smaller than level of precision
        preAlpha              = alpha
        
        while(np.array_equal((x0+alpha*direction),x0)):
            alpha             = alpha*gamma
        
        obj,dx,lossobj,regobj = objGrad(x0+alpha*direction,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
        ogcalls               = ogcalls + 1
        
    return alpha,obj,dx,lossobj,regobj,ogcalls

In [24]:
def backtrack(x0,alpha,direction,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,gamma,obj,dx,lossobj,regobj,digits,obj0,c1,etazero,oldalpha,oldobj,olddx):
    #ensures either (1) Armijo lowex0,alpha,direction,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcallsr objective, or (2) infinitesimal alpha
    doBacktrack               = 0
    preAlpha                  = alpha
    while((pround(obj,digits)>pround(obj0 + c1*alpha*etazero, digits)) and (np.sum((x0+alpha*direction)!=x0)>0)):
        if(ogcalls > 1):
            if((oldalpha > alpha/gamma) and (oldobj < (obj0+c1*oldalpha*etazero))):
                alpha,obj,dx  = restoreAlpha(oldalpha,oldobj,olddx)
                doBacktrack   = 1
                break
        alpha                 = alpha/gamma
        obj,dx,lossobj,regobj = objGrad(x0+alpha*direction,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
        ogcalls               = ogcalls + 1
        doBacktrack           = 1
    return alpha,obj,dx,lossobj,regobj,ogcalls,doBacktrack

In [25]:
def saveAlpha(alpha,obj,dx):
    #make sure current step yields decrease in objective
    oldalpha   = alpha
    oldobj     = obj
    olddx      = dx
    return oldalpha,oldobj,olddx

In [26]:
def restoreAlpha(oldalpha,oldobj,olddx):
    alpha      = oldalpha
    obj        = oldobj
    dx         = olddx
    return alpha,obj,dx

In [27]:
def pround(x,d):
    d          = round(d)
    if (d<1):
        print('Number of digits must be integer d = %.4e \n'%d)
    if(x==0 or math.isnan(x) or math.isinf(x)):
        y      = x   
    else:
        p      = math.floor(math.log10(abs(x)))+1
        factor = 10**(d-p)
        y      = np.round(x*factor)/factor
    return y

In [28]:
def checkConditions(obj,digits,obj0,etazero,eta,x0,alpha,direction):
    if round(obj, digits) >= round(obj0, digits):
        print('Warning: Finished line search without decreasing objective.\nMay have reached limit of precision, or obj/grad code may be broken.\n')
    if etazero == eta:
        print('Warning: Line search yielded no change in directional derivative.\nMay have reached limit of precision.\n')
    if sum(x0 + alpha * direction != x0) == 0:
        print('Warning: Line search yielded no change in position.\nMay have reached limit of precision.\n')

In [29]:
def h(z):
    zin01      = (z>0)^(z>=1)
    zle0       = z<0
    ret        = zin01/2 - zin01*z + zin01*(z**2)/2 + zle0/2 - zle0*z
    return ret  

In [30]:
def hprime(z):
    zin01      = (z>0)^(z>=1)
    zle0       = z<0
    ret        = zin01*z - zin01 - zle0
    return ret

In [31]:
def m3fSoftmax(xy,theta):
    n,m        = xy.shape
    n1,l1      = theta.shape
    if(n!=n1):
        print('sizes of xy and theta don''t match');
    y          = np.ones((n,m))
    for i in range(l1):
        tmp    = ((theta[:,i]).reshape(-1,1))@(np.ones((1,m)))
        tmp    = xy>=tmp
        y      = y + tmp
    return y

## Loading Matrices for Video Games domain

In [32]:
from pandas import read_csv

In [33]:
rating_matrix_video_4k = np.loadtxt("rating_matrix_video_4k.csv", delimiter=",")

In [34]:
temp_d = read_csv('new_user_ids_video_4k.csv')
temp_np = temp_d.to_numpy()
temp_np = np.delete(temp_np, 0, 1)
new_user_ids_video_4k = temp_np.reshape(temp_np.shape[0],)

In [35]:
temp_d = read_csv('item_ids_video_4k_rated_unrated.csv')
temp_np = temp_d.to_numpy()
temp_np = np.delete(temp_np, 0, 1)
item_ids_video_4k_rated_unrated = temp_np.reshape(temp_np.shape[0],)

## Loading Matrics for Movies and TV domain

In [36]:
rating_matrix_mt_4k = np.loadtxt("rating_matrix_mt_4k.csv", delimiter=",")

In [37]:
temp_d = read_csv('new_user_ids_mt_4k.csv')
temp_np = temp_d.to_numpy()
temp_np = np.delete(temp_np, 0, 1)
new_user_ids_mt_4k = temp_np.reshape(temp_np.shape[0],)

In [38]:
temp_d = read_csv('item_ids_mt_4k_rated_unrated.csv')
temp_np = temp_d.to_numpy()
temp_np = np.delete(temp_np, 0, 1)
item_ids_mt_4k_rated_unrated = temp_np.reshape(temp_np.shape[0],)

## Preprocessing Video Games domain

In [49]:
names = ['user_id', 'game_id', 'rating', 'timestamp']
df    = pd.read_csv('ratings_Video_Games.csv', sep=',', names=names)
df.head()

,user_id,game_id,rating,timestamp
0,AB9S9279OZ3QO,0078764343,5.0,1373155200
1,A24SSUT5CSW8BH,0078764343,5.0,1377302400
2,AK3V0HEBJMQ7J,0078764343,4.0,1372896000
3,A10BECPH7W8HM7,043933702X,5.0,1404950400
4,A2PRV9OULX1TWP,043933702X,5.0,1386115200


In [50]:
n_users_video = df.user_id.nunique(dropna = True)
m_items_video = df.game_id.nunique(dropna = True)
print(str(n_users_video) + " " + str(m_items_video))

826767 50210


In [51]:
user_ids_video = df.user_id.unique()
item_ids_video = df.game_id.unique()
print(user_ids_video.shape)
print(item_ids_video.shape)

(826767,)
(50210,)


## Preprocessing Movies & TV domain

In [52]:
names = ['user_id', 'movie_id', 'rating', 'timestamp']
df    = pd.read_csv('ratings_Movies_and_TV.csv', sep=',', names=names)
df.head()

,user_id,movie_id,rating,timestamp
0,A3R5OBKS7OM2IR,0000143502,5.0,1358380800
1,A3R5OBKS7OM2IR,0000143529,5.0,1380672000
2,AH3QC2PC1VTGP,0000143561,2.0,1216252800
3,A3LKP6WPMP9UKX,0000143588,5.0,1236902400
4,AVIY68KEPQ5ZD,0000143588,5.0,1232236800


In [53]:
n_users_mt = df.user_id.nunique(dropna = True)
m_items_mt = df.movie_id.nunique(dropna = True)
print(str(n_users_mt) + " " + str(m_items_mt)) 

2088620 200941


In [54]:
user_ids_mt = df.user_id.unique()
item_ids_mt = df.movie_id.unique()
print(user_ids_mt.shape)
print(item_ids_mt.shape)

(2088620,)
(200941,)


## Dataset trimming

In [55]:
overlap = 4000

In [56]:
print(user_ids_video.shape)
print(user_ids_mt.shape)

(826767,)
(2088620,)


In [57]:
overlapping_user_ids = np.intersect1d(user_ids_video, user_ids_mt)
print(overlapping_user_ids)
print(overlapping_user_ids.shape)

['A0002090WKEMAO8KOWKM' 'A00230923E4Y7VHWZK0IC' 'A002439424KGHR3LZ1OMZ'
 ... 'AZZW55XIQD8QC' 'AZZY72H9Y2F6O' 'AZZZ9DDUPKNKC']
(155797,)


In [58]:
non_overlapping_user_ids_video = np.array(list(set(user_ids_video.tolist()).difference(set(overlapping_user_ids.tolist()))))
non_overlapping_user_ids_mt = np.array(list(set(user_ids_mt.tolist()).difference(set(overlapping_user_ids.tolist()))))
print(non_overlapping_user_ids_video.shape)
print(non_overlapping_user_ids_mt.shape)

(670970,)
(1932823,)


In [59]:
req_overlap_user_ids = np.random.choice(overlapping_user_ids, size=overlap, replace=False)
print(req_overlap_user_ids.shape)
# test = np.array(list(set(overlapping_user_ids_t.tolist()).difference(set(train.tolist()))))

(4000,)


In [60]:
req_non_overlap_user_ids_video = np.random.choice(non_overlapping_user_ids_video, size= 9000 - overlap, replace=False)
req_non_overlap_user_ids_mt = np.random.choice(non_overlapping_user_ids_mt, size= 7000 - overlap, replace=False)
print(req_non_overlap_user_ids_video.shape)
print(req_non_overlap_user_ids_mt.shape)

(5000,)
(3000,)


In [61]:
new_user_ids_video_4k = np.concatenate((req_overlap_user_ids, req_non_overlap_user_ids_video), axis=None)
new_user_ids_mt_4k = np.concatenate((req_overlap_user_ids, req_non_overlap_user_ids_mt), axis=None)
np.random.shuffle(new_user_ids_video_4k)
np.random.shuffle(new_user_ids_mt_4k)
print(new_user_ids_video_4k.shape)
print(new_user_ids_mt_4k.shape)

(9000,)
(7000,)


In [62]:
print(new_user_ids_video_4k)
print(new_user_ids_mt_4k)

['A1RXN43VZ8F3DF' 'A3M5E7IEHJ5FWZ' 'AIDX2XAMLTIHX' ... 'A192NXERQ6IPOL'
 'A1215345O3XF0M' 'A2G38L8CLFUOR8']
['A1ILQ9LXRWDZYE' 'A2O1N4607VUQQL' 'A27C2AKSMGAN0I' ... 'AMQQHEASTMA0D'
 'A27B64PWDUBQ8Y' 'A3PZX07OBTWLLO']


In [63]:
overlapping_user_ids_t = np.intersect1d(new_user_ids_video_4k, new_user_ids_mt_4k)
print(overlapping_user_ids_t)
print(overlapping_user_ids_t.shape)

['A0108605182MOPK0I9YBV' 'A0369367M6U3LSLAB3C1' 'A05299163KOLQAXMGQF3W'
 ... 'AZYMH3ZDZML7C' 'AZZCVWF0H0O5Q' 'AZZJX02B2F22D']
(4000,)


## Create rating matrix for video games domain

In [64]:
names = ['user_id', 'game_id', 'rating', 'timestamp']
df    = pd.read_csv('ratings_Video_Games.csv', sep=',', names=names)
df.head()

,user_id,game_id,rating,timestamp
0,AB9S9279OZ3QO,0078764343,5.0,1373155200
1,A24SSUT5CSW8BH,0078764343,5.0,1377302400
2,AK3V0HEBJMQ7J,0078764343,4.0,1372896000
3,A10BECPH7W8HM7,043933702X,5.0,1404950400
4,A2PRV9OULX1TWP,043933702X,5.0,1386115200


In [65]:
df = df.loc[df['user_id'].isin(new_user_ids_video_4k.tolist())] 

In [66]:
df.shape

(16897, 4)

In [67]:
present_rating = df.rating.unique()
print(present_rating)

[3. 5. 4. 2. 1.]


In [68]:
new_item_ids_video_4k = df.game_id.unique()
print(new_item_ids_video_4k.shape)
print(new_item_ids_video_4k)

(8336,)
['0439394422' '0700026657' '0700099867' ... 'B00KKAQYXM' 'B00KN7AZH8'
 'B00LBAM588']


In [69]:
print(item_ids_video.shape)

(50210,)


In [70]:
item_ids_video_4k_rated_unrated = new_item_ids_video_4k
np.random.shuffle(item_ids_video_4k_rated_unrated)
print(item_ids_video_4k_rated_unrated.shape)

(8336,)


In [71]:
rating_matrix_video_4k = np.zeros([9000, item_ids_video_4k_rated_unrated.shape[0]], dtype='float32')
print(rating_matrix_video_4k.shape)

(9000, 8336)


In [72]:
for ind in df.index:
    if df['user_id'][ind] in new_user_ids_video_4k:
        rating_matrix_video_4k[np.where(new_user_ids_video_4k == df['user_id'][ind])[0][0]][np.where(item_ids_video_4k_rated_unrated == df['game_id'][ind])[0][0]] = df['rating'][ind]
        print(str(df['user_id'][ind]) + " " + str(df['game_id'][ind]) + " " + str(df['rating'][ind]))

A1TL721YECDIM8 0439394422 3.0
A3HR8L31S8AFF6 0700026657 5.0
A5GEGX2JHHDC0 0700099867 4.0
A96K1ZGW56S2I 0758534531 5.0
A22GUJTK1DM4BL 1886846820 5.0
AGKHQA6OATS1E 3828770193 3.0
A4TRW9IKMK66A 3866811659 2.0
A2Q5ZDVPELFG2U 7100027950 5.0
A3QGL4W4KC6I45 7293000928 3.0
AOS2A9LDAP1ZJ 7293000936 5.0
A2BCP0H3RW6VMU 7543450933 5.0
A1XVGYO4X0JSOA 9576754267 5.0
A1SN8NEULE5JFO 9625990674 3.0
A1RTR7DAEFHL1L 9625990992 5.0
AX2BTRU44O9B6 9861019731 5.0
A3EMFUH9LVFL6 9882155456 4.0
A35L6ZK8RSG75H B000006OTC 5.0
AACIKO4GH062J B000006OVE 1.0
AG8PGHW1MYCIY B000006OVE 4.0
A3OLUM9IOWUPD5 B000006OVK 5.0
A2204RPNZSYO6J B000006OWS 5.0
A3JDPG0KS3JIYY B000006P0K 5.0
A30BWPKM0R22K0 B000006P0M 5.0
A28FCUZVFFXPXM B000006RGR 5.0
A28FCUZVFFXPXM B000006RGS 5.0
A3NKJSCTLZ6YV5 B000007NJE 1.0
A1XGTERS7UVRU5 B000007NJE 3.0
A2ICCH1MRJL2QD B000009QCV 4.0
A1TR4Q13KV1CW B000009QCZ 4.0
A3RQLBLI6LNCXT B00000DMA8 5.0
A2K6F6W0TLXE8H B00000DMAA 3.0
A2ZV00GFXSG33Z B00000DMAA 5.0
A24NLJBQO3D54 B00000DMAA 5.0
A3KMHVAIOS9WFV B00000

A2RUGMW42P0L1H B00008KA4E 2.0
A2S18BY2R3B64J B00008KA4E 5.0
A30PJIX2O6TQZZ B00008KA4E 1.0
A1157AP690MY54 B00008KTNW 4.0
A2BUS6GUVDV1WG B00008KTNW 5.0
A37DLZMMT4AKEG B00008KTNW 5.0
A31KOU3P4U7Z89 B00008KTNW 3.0
A3MKU0EI647E1V B00008KTNW 5.0
A3028S6C0F8BF0 B00008KTNW 1.0
A2RFYZM8VQ1LFN B00008KTRS 4.0
AQ6F5FMEFRIKL B00008KTSP 5.0
AP73WC2NKEC9I B00008KTSW 3.0
A3MOP8C94HTZHO B00008KTSW 5.0
A2W2XSFA9YGJN B00008KTW2 4.0
A3LQ4FESP2A7BD B00008KTWG 5.0
A2XNENW0TEG516 B00008KU9T 5.0
AFPH2BZGNNJ79 B00008KU9Z 5.0
AMNCE2PCFP4XU B00008KU9Z 5.0
A1WMIJPDZ6FJBM B00008KUA8 2.0
AV6RDHTAINL4R B00008KUAB 4.0
A21AEWBOLO6Q2N B00008KXG5 3.0
A2IP3ZK11RY3NW B00008KXG5 3.0
A1U5292AFT6AU6 B00008KXG5 4.0
A1Q5N8XHR2SRLW B00008NRM7 4.0
A11U1VA2GG3GGW B00008NRM8 4.0
A2BWT9OLAM03XI B00008NRMB 4.0
A3L14D8BAREUSV B00008OD54 5.0
AHZNYYVLZ50U2 B00008ODBQ 4.0
A2WM1KL9DQSQ18 B00008OE4Z 3.0
A3VV4KOF0DR3HY B00008PX7K 5.0
AU33330HFABD B00008RGQL 3.0
A1SUALPMT2B6GP B00008RUYZ 5.0
A1M31S9MQ4B6P5 B00008RUYZ 5.0
A2TLDZ72PJCGZ3 B000

A18NFQFUQ4KF68 B00FATRKOK 5.0
A25WPAO3L8TVJZ B00FATRKOK 5.0
A1WEW9SDFATIVD B00FATRKOK 5.0
A1THS0DHF0TFKX B00FATRKOK 5.0
A3R01WHD75L6FG B00FATRKWM 5.0
A3EA5JN76TMQ8U B00FAVVKO4 2.0
A2DTYJ57JTJ8TQ B00FAX6XQC 5.0
A27Q5VRICLLMPB B00FAX6XQC 5.0
A1FCKFWPWIPZRA B00FAX6XQC 4.0
ARSJAPQR6H4X7 B00FAX6XQC 5.0
A3U9OKWB9EJ47O B00FAX6XQC 5.0
A2QAUD97PVO6KK B00FAX6XQC 5.0
A1H5JVFZNUAQGI B00FAX6XQC 3.0
A2RWKKE0X5TMWC B00FAX6XQC 5.0
A1S725MQD0ZSCF B00FAX6XQC 3.0
AVD5QQ90F65RU B00FAX6XQC 1.0
A1I3JMMPVHCC04 B00FAX6XQC 1.0
A3THFL1HC4MWIF B00FAX6XQC 5.0
A2WYAQV76LDWR3 B00FAX6XQC 5.0
A255Q2PEE5R5J8 B00FAX6XQC 1.0
A1986VFZHBL066 B00FAX6XQC 5.0
A2D6HYCV0DF1OW B00FAX6XQC 5.0
A16KH6OYYGKCWO B00FAX6XQC 5.0
A2ERKTPRKP048Z B00FAX6XQC 5.0
A16ABYF1XQZUL8 B00FAX6XQC 5.0
A14CLUBO70I3SV B00FAX6XQC 4.0
A21IFKXE4H0HR6 B00FAX6XQC 5.0
ALGPV4AMBG54P B00FAX6XQC 5.0
A3NIZ5DBFKU3GZ B00FAX6XQC 2.0
A2712L6FFYWXQ7 B00FAX6XQC 5.0
A1J3IHLSEMKVXL B00FAX6XQC 5.0
A3OVA61Q90FGJT B00FAX6XQC 5.0
A2HY0W97LC08Y8 B00FAX6XQC 5.0
A379RFM0LCL56

In [73]:
rating_matrix_video_4k

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [74]:
unique, frequency = np.unique(rating_matrix_video_4k, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

Unique Values: [0. 1. 2. 3. 4. 5.]
Frequency Values: [75007103     1766      951     1682     3485     9013]


## Creating rating matrix for movies and TV domain

In [75]:
names = ['user_id', 'movie_id', 'rating', 'timestamp']
df    = pd.read_csv('ratings_Movies_and_TV.csv', sep=',', names=names)
df.head()

,user_id,movie_id,rating,timestamp
0,A3R5OBKS7OM2IR,0000143502,5.0,1358380800
1,A3R5OBKS7OM2IR,0000143529,5.0,1380672000
2,AH3QC2PC1VTGP,0000143561,2.0,1216252800
3,A3LKP6WPMP9UKX,0000143588,5.0,1236902400
4,AVIY68KEPQ5ZD,0000143588,5.0,1232236800


In [76]:
df = df.loc[df['user_id'].isin(new_user_ids_mt_4k.tolist())] 
df.shape

(24778, 4)

In [77]:
new_item_ids_mt_4k = df.movie_id.unique()
print(new_item_ids_mt_4k.shape)
print(new_item_ids_mt_4k)

(13729,)
['0001527665' '0307142469' '0307142485' ... 'B00KSON3L6' 'B00L4IDS4W'
 'B00LH9ROKM']


In [78]:
print(item_ids_mt.shape)

(200941,)


In [79]:
item_ids_mt_4k_rated_unrated = new_item_ids_mt_4k
np.random.shuffle(item_ids_mt_4k_rated_unrated)
print(item_ids_mt_4k_rated_unrated.shape)

(13729,)


In [80]:
rating_matrix_mt_4k = np.zeros([7000, item_ids_mt_4k_rated_unrated.shape[0]], dtype='float32')
print(rating_matrix_mt_4k.shape)

(7000, 13729)


In [81]:
for ind in df.index:
    if df['user_id'][ind] in new_user_ids_mt_4k:
        rating_matrix_mt_4k[np.where(new_user_ids_mt_4k == df['user_id'][ind])[0][0]][np.where(item_ids_mt_4k_rated_unrated == df['movie_id'][ind])[0][0]] = df['rating'][ind]
        print(str(df['user_id'][ind]) + " " + str(df['movie_id'][ind]) + " " + str(df['rating'][ind]))

A3478QRKQDOPQ2 0001527665 5.0
A2GPA8G4EJX2N5 0307142469 5.0
A2VSM3PGE1ZJ31 0307142485 5.0
A1XGTERS7UVRU5 0307514161 5.0
A5V62IILGANFN 0307732207 5.0
A1Q9U88AP5DMA0 0310257158 3.0
A3EC537ARUVP7D 0310263662 4.0
A3RB6NQOLOBXYD 0310263662 5.0
A3AAX9AO0CIFBI 0310263662 4.0
A2DAHERP7HYJGO 0310263662 5.0
AIRHRV3D8X4J1 0310263662 5.0
AER15RIMV8E6D 0310263662 4.0
A2UO4Y0063LNBX 0310263662 4.0
A1MOXJOSJ6A2KW 0310263662 5.0
A96K1ZGW56S2I 0310263662 5.0
A1324FWI7LOIGR 0310263662 5.0
A96K1ZGW56S2I 0310269164 5.0
A2L29MWPWG6LG4 0310285569 5.0
A22L4O7MSTRQ56 0321535154 5.0
A96K1ZGW56S2I 0615115187 5.0
A2QK0CLGM4Q0R3 0615115187 5.0
A1AD6Q44FLXYD1 0681396474 5.0
A3ENU1YBCW5MGI 0718000315 4.0
A2XFFNEPR8VV5S 0736915842 2.0
A1LPST4JIBM701 0738920371 5.0
A1TRIHFCHSFKJC 0738921351 5.0
A1JHCSJLJA3M21 0739040308 1.0
A27ZOCD5B63Y0P 0739600443 1.0
A20ROFUIT7HO6P 0740318764 4.0
A377HG5PIJZHUC 0740318934 5.0
A2A8D0QPWG5XU 0740328271 3.0
A14JNK14Q9GDHG 0758907125 5.0
ATGDJOCF60QOU 0764001035 5.0
AIKLFAHS45LUE 0764

A12UL82C20SCG3 078062856X 3.0
A1NLDMELNDFORP 0780628799 5.0
A334BRCOLME8BE 0780628799 5.0
A1WLTLB9LAKCHG 0780628799 4.0
A15TIGL6MKUWYU 0780628799 5.0
A12A8LU93A132U 0780628799 4.0
AIRHRV3D8X4J1 0780629159 5.0
A2SNCT2EK1TY28 0780629159 5.0
A25JFM76WICHK6 0780629450 5.0
A1NGSFS3OACX0J 0780629450 5.0
A2J0Y71BZ13ISW 0780629450 5.0
A1QOSRG9U2J6F1 0780630459 5.0
AMZYCMVLCC57G 0780630459 5.0
A1X1BPPG5HZXZH 0780630858 3.0
AER15RIMV8E6D 0780630858 3.0
AER15RIMV8E6D 0780630866 4.0
AER15RIMV8E6D 0780630874 4.0
AER15RIMV8E6D 0780630882 5.0
AER15RIMV8E6D 0780630890 4.0
AIRHRV3D8X4J1 0780630904 5.0
AER15RIMV8E6D 0780630904 4.0
A2A7CI7OWBOI3A 0780630904 3.0
A2PIGWTMC5V889 0780631153 5.0
A13C6NTJ3HO0FX 0780631153 4.0
A14SSWB3ZU6BQR 0780631153 4.0
AA8I85W0KCTPZ 0780631153 3.0
A2U7Z7RX0HVY7G 0780631153 2.0
A3LULFOYWUK2SA 0780631234 5.0
A2D7WLYMV2738P 0780631234 4.0
A2D7WLYMV2738P 0780631684 5.0
A15TIGL6MKUWYU 0780631684 2.0
A20DYINAIC0UVB 0780631684 5.0
A2RII3OGS8LJJ4 0780651359 4.0
ADUGOXKRC2A2D 078065

A1CRSSHIZ5MWIQ 1582704414 5.0
A30CP8E6AZLGNV 1582704414 5.0
A32S2SF5U73GZR 1582704414 2.0
AWPODHOB4GFWL 1582704414 4.0
A2SRVDDDOQ8QJL 1582704414 2.0
AWZ88AO0F0J90 1582704414 5.0
A3IE4ONEZPOLPX 1584680598 4.0
ARPTJEKQNOBWJ 1586551027 5.0
A1PUE4V0SE0YKB 1586640046 2.0
A2AXNXBHIGHCDL 1586640046 5.0
A36G7YACJDMN4Y 1586683950 5.0
A2T6M6S3AKG0UI 1589977238 4.0
A3FBTJCCNQXW2U 1591259886 4.0
A23534DSBEMVCH 1591791847 5.0
AMD8IR57P6R5V 1598037994 5.0
A1HXVE4Q20DM39 1598802291 5.0
ANJ66KFXM0SW8 1600850774 5.0
A3C8JK7L5BH0AK 160142311X 5.0
AXNG7TKB7FUXK 1603620494 4.0
A18KJVYDCQK7EH 1604074396 5.0
A3LPNWHQ0II0YW 1605296295 5.0
A283BOGS9IO7M7 160529893X 5.0
A3NG9ADCTVRV3O 1608838137 4.0
AD8OV2JFYGDA 1608838137 5.0
A2AUGLAVFSPVSP 1608838137 5.0
A3DS6OGY2XODTK 1608838137 4.0
A3H21Z5YYUI7L2 1608838137 5.0
A39AK7K7ONA53Z 1608838137 5.0
A2XP4QPWALO5Z5 1608838137 5.0
A35R3N92G7UMEO 1608838137 5.0
A27R7GV0I01HGN 1608838137 5.0
ADTIJ5KHUCAND 1608838137 5.0
A1ZAMCVLZ4RQZL 1608838137 5.0
A3MUII6MG5EEEN 1608

A3UAWC69O4HR03 6302020190 5.0
A10LAX6QGJD4PP 6302020190 2.0
A1MB35OXDXCMNE 6302020816 4.0
A13IRQRYIA1X71 6302021588 5.0
A35LWA8VY66G8F 6302023777 5.0
A3IPB87ORLH4PA 6302025079 4.0
A1Z9EWIA4KHUZR 6302027748 5.0
A3U3BBFRAMECZ2 6302032237 5.0
AHDBEIR9YT1QB 6302033381 5.0
A96K1ZGW56S2I 6302035058 2.0
APQ8N1UKS1RDC 630203518X 4.0
ADQ30FEBENTUI 6302035325 5.0
A303UW0QGBB7OC 6302035686 4.0
A1GGJB0S7AS00O 6302036720 2.0
APQ8N1UKS1RDC 6302036755 3.0
A2TPDK0KZLHHG5 6302037123 5.0
A39T7F7239R7P5 6302037123 4.0
APQ8N1UKS1RDC 6302037840 2.0
A1VL17WROI088S 6302038308 1.0
ALHNJGPDV4EC0 6302038308 5.0
A1NFUK1XNKRR8Y 6302038308 4.0
A188JKV8QVVOT7 6302038308 5.0
A3RB6NQOLOBXYD 6302038308 5.0
ANLYUPUAUZ81U 6302038308 5.0
A3AKPY4LXIC24O 6302038308 5.0
A96K1ZGW56S2I 6302038308 4.0
ARFPGL7OEATYH 6302038308 4.0
A3D3L8R0OW7WX6 6302038324 4.0
A3IE6Q6KZTT343 6302038324 4.0
A36AXJT8ACQZ5T 6302038448 4.0
A1L7EL9EFHO71R 6302053641 5.0
A1L7EL9EFHO71R 630205365X 5.0
A1L7EL9EFHO71R 6302053668 5.0
A28RCAA27DE22E 63020

A2E3HCZ5D2JZAI B00004WIBC 5.0
A13IRQRYIA1X71 B00004WICW 5.0
APNLFE7D7ZQZA B00004WICW 5.0
A25UGTJJ5DGVE2 B00004WMDT 5.0
AIRHRV3D8X4J1 B00004WZZN 5.0
A2WUWAVZF4WCJE B00004X13Q 5.0
AUU9YT28WCGBW B00004X13Q 5.0
A15TIGL6MKUWYU B00004X13Q 4.0
A2A1HDK1NXG1S2 B00004XMRY 3.0
A96K1ZGW56S2I B00004XMS2 5.0
A3H7U1AI050C6U B00004XMVA 5.0
A1EBRLPOKCQJO4 B00004XMVA 5.0
A1YKAJYECYYYUZ B00004XOR3 5.0
A37KS7CG33BBH3 B00004XPPF 5.0
A1AOMTP8YKG9OX B00004XPPF 5.0
A43TV4W6F5SA B00004XPPG 4.0
A16O5CC1YTZWEE B00004XPPG 1.0
A237W2TMB5S09J B00004XPPG 5.0
A12A8LU93A132U B00004XPPG 4.0
A2RB67N8XJ52QB B00004XPPG 5.0
A37NT5U3WXL1XX B00004XQMU 5.0
APQ8N1UKS1RDC B00004Y3WB 4.0
A3KI9U3HED7OY1 B00004Y56F 2.0
A2A1HDK1NXG1S2 B00004Y56F 5.0
AER15RIMV8E6D B00004Y62X 4.0
A1XGTERS7UVRU5 B00004Y62Z 4.0
A19OKQ1NP8IR1D B00004Y631 1.0
AA1HGLBR664VQ B00004Y631 5.0
A96K1ZGW56S2I B00004Y631 1.0
A3D3J409VE0D4X B00004Y634 1.0
AWPT31A3YL590 B00004Y6A5 3.0
A1TW5TBQHER12D B00004Y6AE 4.0
AC66JNFP2W8FD B00004Y6AN 5.0
A27ZOCD5B63Y0P B00004Y

...

A2UF7U3J3QXX3Y B001GLX74I 5.0
AWPODHOB4GFWL B001GM0GN2 5.0
AER15RIMV8E6D B001GMAVGE 5.0
AVQ4CY0QZDVBR B001GMAVGE 3.0
A96K1ZGW56S2I B001GMAVGE 4.0
A3KQI3NNN88YN0 B001GMH8WE 1.0
A2PXDO4WSOSHS9 B001GMH8WY 5.0
A15V61R3F2D11L B001GN3ZJI 2.0
A3NWWXWW3XDSZ2 B001GP5TLI 2.0
A1FM7E2U6FC93B B001GP5TLI 5.0
A1561KYDLREXIK B001GP5TLI 5.0
A2WUMUR47EQDHG B001GP5TLI 5.0
A1X2UI24JY8620 B001GP5TLS 5.0
A2TVM0L66ZSB0B B001GP5TLS 5.0
A3R4Y2G0NH2PJ4 B001GP5TLS 5.0
ANCP7YSBL65VT B001GP5TMM 1.0
A38J9KTPWDG6SI B001GP5TOU 3.0
A2YBNGAGOL4ODO B001GQ3ECI 5.0
A3HHJMGE3M8MD4 B001GU04LI 4.0
A22VV7GFUDUZ3C B001GUL6BK 5.0
AHFNHKU7190YN B001GUL6BK 5.0
A23RLZ29VIJMF5 B001GUL6BK 5.0
AM0V43YEA1JBR B001GUMOXE 5.0
AER15RIMV8E6D B001GZM46G 3.0
A35R3N92G7UMEO B001H1GMBC 4.0
A2RCG71URVEQ5D B001H1GMBC 5.0
AER15RIMV8E6D B001H5X6SA 5.0
A1B68LH2GNC73M B001H5X6SK 5.0
A2DY6I9MVQ7FDC B001H5X7I4 3.0
A5NW620UKE9K B001H5X7I4 3.0
AER15RIMV8E6D B001H5X7I4 4.0
A2KCNRFE5NL5B6 B001H5X7I4 5.0
A3478QRKQDOPQ2 B001H8BU4Y 5.0
A4G8Z3YMBLN0F B001H9JB

A3AKVVJ0TIOG5N B001MBUGLY 3.0
A116WYUUH8SGWX B001MBUGLY 5.0
AZAC8O310IK4E B001MBUGLY 3.0
A1QJH4IWFC6B4R B001MBUGLY 4.0
A2ZSM2ISJJX7EY B001MBUGLY 3.0
A1VWGQUGN6AYBN B001MBUGLY 5.0
A2A31O02647RQS B001MBUGLY 4.0
A13GYZ0KXQS98V B001MEJTU0 4.0
A78F45HMYSBEC B001MEJY8W 5.0
AKH4BTY3E72VJ B001MEJY8W 3.0
AHS2Y4XQCAQ2R B001MEJY9G 5.0
AXFQVNN17LJU9 B001MEJYBO 5.0
A13WBUX01Q4D35 B001MEWBVE 4.0
A30NQSX4J2YWJB B001MFNB4E 5.0
AK9UPC78LIPIR B001MFNB4E 3.0
A38U76FKWE0EA7 B001MFNB4O 5.0
A1N10E26UAQ2TF B001MFNB4O 5.0
A14QWUG2NSAH9I B001MK8GUI 5.0
A3HBVV942IKONH B001MLZNKS 5.0
A5NW620UKE9K B001MLZNKS 2.0
A5NW620UKE9K B001MQD2ZG 3.0
A2CPEG4JYUT2N B001MSXQ2S 3.0
A20MXB65JKHMNC B001MT7ZEW 5.0
A1WZK1FN3S355V B001MT7ZEW 5.0
ATX06IWCT2KC3 B001MT7ZEW 1.0
AER15RIMV8E6D B001MT7ZEW 5.0
A38BIFAA5HDL8N B001MT7ZI8 1.0
A2REIU20P3S7SG B001MTEXS8 5.0
A2DAHERP7HYJGO B001MTYRJS 3.0
AER15RIMV8E6D B001MTYRJS 3.0
AS57W89S5RK5R B001MVYUQG 5.0
A1RV3ITFIDCKI6 B001MVYUQG 5.0
A2DAHERP7HYJGO B001MVYUQQ 5.0
AMUP8DYE7EAN2 B001MVYUQQ 

A1WOW1Q6AODKYL B0031RAOVY 5.0
A3TZJJKWDKSDFI B0031RAOVY 5.0
A1OZLTGEPO7IAB B0031RAOVY 5.0
A26HA770BNVZIT B0031RAP5Y 4.0
A3J121CFA2DC0T B0031REQJ0 5.0
A3ONTLXQ85EE4G B0031REQJ0 4.0
A2WUWAVZF4WCJE B0031U8OEK 5.0
A3RRERNL7LGDFL B0031XYLWG 5.0
A1YLN0TYI96QQP B0031XYLWG 5.0
A1MCJB8YY38G7S B0031XYLWG 5.0
A1U4QUTLTMWRY0 B0031XYLWG 5.0
A3PIJVA4U0NDFD B0031Y8018 5.0
A2CAX0VE8G4LBS B0032AM8QE 5.0
A20Y02G77U9KKY B0032EFQQE 5.0
A2C9T4SNJAPD59 B0032JTV38 5.0
A1MOXJOSJ6A2KW B0032JTV38 5.0
A31X95ZOJEBHM9 B0032JTV38 5.0
A2FJCSWEV5JNED B0032JTV38 2.0
A2VXMPDC2A5MYK B0032JTV6A 4.0
AER15RIMV8E6D B0032JTV6A 4.0
A29ZTHN740W371 B0032LV4KY 5.0
A3RVW73IX5CDSL B0032LV4MM 5.0
A1WC7Q4BO2DQOU B0032LV4MM 4.0
A25M4WQPKSPY5M B0032O8B7K 5.0
A2V37R7ZGLU9M9 B0032OE6O2 4.0
A2CDJ168P3S12M B00330JPVY 5.0
A3DU9EC223TCQS B00331RHCM 4.0
A3DU9EC223TCQS B00331RHPO 4.0
AMONGV0LAI1O7 B00332WQZ4 5.0
AJLYAY1G5Z8GE B0033AGJ5Y 5.0
A3NL1KW56O3JR8 B0033AGJ5Y 3.0
AJLYAY1G5Z8GE B0033HKDXG 5.0
A3BR3IV6THTYXO B0033HWZS2 3.0
AU27QDF5GYH1T 

A334BRCOLME8BE B003EYVXV4 5.0
A1RYQWVB56HDLZ B003EYVXV4 5.0
A2G4HRAHJ2W92Y B003EYVXV4 4.0
A10ZPZYYV8N29J B003EYVXV4 4.0
A18DV4PXIUV4BN B003EYVXV4 4.0
A1WTOHZV2DOFBR B003EYVXV4 4.0
A4MA0NVC6A7XC B003EYVXV4 3.0
A3L1192BT30QPT B003EYVXV4 5.0
A35N9IVGHC366X B003EYVXV4 5.0
A2XNFVXOOWEFUM B003EYVXV4 4.0
A1QMVJ15GHL34R B003EYVXV4 5.0
AK50PJ8B6J5XW B003EYVXV4 4.0
A229XJHG06OKS6 B003EYVXV4 5.0
A3S2EXYIHA64IO B003EYVXV4 5.0
AZXQDG6VKP09I B003EYVXV4 5.0
A1WUSQNTO1ND7W B003EYVXV4 4.0
A1MKNB1WADTR5X B003EYVXV4 5.0
A30RLQXHQQ7XB9 B003EYVXV4 3.0
A3RS2144WMQWV0 B003EYVXV4 4.0
A2WIIBXBTDWQQ B003EYVXV4 3.0
A2DAHERP7HYJGO B003EYVXV4 4.0
A2L3R1SQPCT8MV B003EYVXV4 3.0
AZ28PD3V6FU1X B003EYVXV4 4.0
A1ZY7S9OUX2BO7 B003EYVXV4 5.0
A1F3CSN4FBKY42 B003EYVXV4 2.0
A3AVVMEI28N2UJ B003EYVXV4 5.0
A2XOV3ZKGDK55G B003EYVXV4 4.0
A1MOXJOSJ6A2KW B003EYVXV4 1.0
A3B3CSHFCPT0I5 B003EYVXV4 4.0
A1M3NVR8A3O627 B003EYVXV4 5.0
AYE8Q1UUL6CUK B003EYVXV4 3.0
A2W7923338BSKJ B003EYVXV4 1.0
A3DERHQVWVNTVY B003EYVXV4 4.0
A2UE9D1TQ3XGUH B

A39GD6AGMNW4QF B007AAF3XC 2.0
A8SEHGWKTX8C4 B007AFBZ4I 5.0
A1WVA7V02PQOY6 B007AJOI0C 5.0
A2PA8HBLW1YH2E B007BMIG7Y 5.0
A2OW86CTD2JPEN B007BMIGQ0 5.0
A1WVA7V02PQOY6 B007BNDET8 5.0
ATGDJOCF60QOU B007BNDFNS 5.0
A2CFI63GSYL397 B007BRVVX0 4.0
A1OJ0ZUXL5X1JV B007BS38J4 5.0
A1UMHCRVZQWQER B007C3TVEY 5.0
A2DAHERP7HYJGO B007C4ZJ0S 3.0
A1WVA7V02PQOY6 B007CBWWF6 5.0
A3IJEQPGZ5BGC7 B007CBWWF6 5.0
A2DAHERP7HYJGO B007CBX548 4.0
A2XR121MR4HGH1 B007CF6S1Q 2.0
AL8QYG1W0PBMS B007CJL16Y 5.0
A2DAHERP7HYJGO B007CO5FFC 5.0
ATDUCMDFRHJ2I B007CO5FFC 1.0
A11U1VA2GG3GGW B007CU8OUE 5.0
AMY5FH3WYKZQT B007CZ31FM 3.0
A2DAHERP7HYJGO B007CZ31FM 1.0
A2DAHERP7HYJGO B007CZ334Q 3.0
A2DMM4YRMWNQ4V B007DIZQQ0 5.0
A126OMDJSQUVMK B007EQR0D8 5.0
A1MOXJOSJ6A2KW B007EQR0D8 5.0
A1M3NVR8A3O627 B007EQR0D8 5.0
A1WVA7V02PQOY6 B007FDOAF6 5.0
A5FQ1XLEQTFE6 B007FDTAKG 4.0
A34G8TQYJ6OSMC B007FGXVCG 3.0
A3KB1L8DIDDTO5 B007FQXEQO 5.0
A1STLRBYMSANTQ B007FSEAHY 1.0
A28P7BQQ27ZQWN B007FSEAHY 1.0
A1QFHAJP6X8MCB B007FZDXWK 5.0
A2WSC4334A91G B0

In [115]:
unique, frequency = np.unique(rating_matrix_mt_4k, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

Unique Values: [0. 1. 2. 3. 4. 5.]
Frequency Values: [94943752     1809     1449     2865     5025    14100]


In [116]:
numpy.savetxt("rating_matrix_video_4k.csv", rating_matrix_video_4k, delimiter = ",")

In [117]:
temp = pd.DataFrame(new_user_ids_video_4k)
temp.to_csv("new_user_ids_video_4k.csv")

In [118]:
temp = pd.DataFrame(item_ids_video_4k_rated_unrated)
temp.to_csv("item_ids_video_4k_rated_unrated.csv")

In [119]:
numpy.savetxt("rating_matrix_mt_4k.csv", rating_matrix_mt_4k, delimiter = ",")

In [120]:
temp = pd.DataFrame(new_user_ids_mt_4k)
temp.to_csv("new_user_ids_mt_4k.csv")

In [121]:
temp = pd.DataFrame(item_ids_mt_4k_rated_unrated)
temp.to_csv("item_ids_mt_4k_rated_unrated.csv")

## Removing overlapping test user_ids from target domain

In [39]:
d                        = 100 

In [40]:
overlapping_user_ids_t = np.intersect1d(new_user_ids_video_4k, new_user_ids_mt_4k)
print(overlapping_user_ids_t)
print(overlapping_user_ids_t.shape)

['A03382832SKK7Q3UMJ6L1' 'A036041773RWH8ILUKBS' 'A036147939NFPC389VLK' ...
 'AZY0WAYW61R3Z' 'AZYJG5MEQ7P1B' 'AZZMDW27MUJR6']
(4000,)


In [41]:
np.random.seed(5)
train = np.random.choice(overlapping_user_ids_t, size=math.ceil(0.8*overlapping_user_ids_t.shape[0]), replace=False)
test = np.array(list(set(overlapping_user_ids_t.tolist()).difference(set(train.tolist()))))

In [42]:
X_train = np.empty((0,100))
Y_train = np.empty((0,100))
X_test = np.empty((0,100))
Y_test = []

In [43]:
for user_id in test:
    ind1 = np.where(new_user_ids_mt_4k == user_id)
    Y_test.append(rating_matrix_mt_4k[ind1][0])
    new_user_ids_mt_4k =np.delete(new_user_ids_mt_4k, ind1, 0)
    rating_matrix_mt_4k = np.delete(rating_matrix_mt_4k, ind1, 0)

In [44]:
Y_test = np.array(Y_test)
print(Y_test.shape)

(800, 13567)


In [45]:
print(new_user_ids_mt_4k.shape)
print(rating_matrix_mt_4k.shape)

(6200,)
(6200, 13567)


## GRS for Video Games domain

In [46]:
epochs                   = 3
tstPer                   = 30                               # Testing Percentage
NoTstIFromG              = 50
l                        = 5                                # Rating Level

K1                       = 10                              #Number of Clusters in User Space
K2                       = 22                              #Number of Clusters in Item Space

d                        = 100                              # latent space size
minUserPerForSel         = 20                               #Item will be select for test only if minUserPerForSel percentage of users in a group has rated
k                        = 40                               # eval@k: precision@k, recall@k
cutoff                   = 3                                # Relevant > cutoff

lambda1UnifiedLS         = 1
lambda3UnifiedLS         = 1

c2                       = 10**-2
tol                      = 10**-3;
maxiter                  = 500;

In [47]:
n, m = rating_matrix_video_4k.shape
print(n, m)

9000 8843


In [ ]:
#Spectral Clustering
simU                     = userSim(rating_matrix_video_4k)
simU                     = (simU + simU.T)/2
UGidx_video              = spectralClu(simU,K1).reshape(-1,1)

simI                     = userSim(rating_matrix_video_4k.T)
simI                     = (simI + simI.T)/2
VGidx_video              = spectralClu(simI,K2).reshape(-1,1)

In [132]:
print(UGidx_video)
print(UGidx_video.shape)

[[1]
 [1]
 [1]
 ...
 [8]
 [9]
 [1]]
(9000, 1)


In [133]:
print(VGidx_video)
print(VGidx_video.shape)

[[0]
 [0]
 [0]
 ...
 [0]
 [2]
 [0]]
(8843, 1)


In [134]:
unique, frequency = np.unique(UGidx_video, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

print()

unique, frequency = np.unique(VGidx_video, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

Unique Values: [0 1 2 3 4 5 6 7 8 9]
Frequency Values: [ 259 6009  320  316  290  313  329  371  424  369]

Unique Values: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
Frequency Values: [5860  167  171  142  139  168  150  121  149  171  153  169  126  125
  143  139  135  125  149  155   50  136]


### Recommendation

In [135]:
np.random.seed(0)
vini = np.random.randn((n*d + m*d + K1*d + K2*d), 1)

In [136]:
# Group Recommendation: Unified LS
objGrad             = gradUnifiedLS
v,numiter,ogcalls,J = conjgrad(vini,objGrad,c2,tol,maxiter,l,d,rating_matrix_video_4k,lambda1UnifiedLS,lambda3UnifiedLS,K1,K2,UGidx_video,VGidx_video)
U_video                   = v[0:n*d].reshape(n,d,order='F')
V_video                   = v[n*d:n*d+m*d].reshape(m,d,order='F')
UG_video                  = v[n*d+m*d:n*d+m*d+K1*d].reshape(K1,d,order='F')
X                   = U_video@V_video.T
X[X<=1]             = 1
X[X>=5]             = 5
XPerA               = aggregratePrediction(X,UGidx_video)
XPerAMat            = XPerA[UGidx_video.reshape(-1,),:]

XPer_Grp                    = UG_video@V_video.T
XPer_GrpMat                 = XPer_Grp[UGidx_video.reshape(-1,),:]
XPer_GrpMat[XPer_GrpMat<=1] = 1
XPer_GrpMat[XPer_GrpMat>=5] = 5


 3741077.6984
1

 2996864.8808
2

 1914610.8644
3

 1844331.6511
4

 1738380.1132
5

 1697732.7356
6

 1688141.0868
7

 1627892.8257
8

 1611508.6693
9

 1489921.5482
10

 1391213.5557
11

 1378538.2886
12

 1315288.6113
13

 1305822.5485
14

 1273861.4088
15

 1252969.2299
16

 1212924.0841
17

 1148916.3527
18

 1138809.2408
19

 1098075.1553
20

 1089532.0408
21

 1056655.6795
22

 1048979.5107
23

 1002432.5890
24

 974297.8650
25

 935525.2566
26

 923180.0932
27

 908090.4018
28

 898097.8723
29

 891060.0554
30

 862455.9233
31

 841016.5726
32

 808765.8235
33

 727496.4295
34

 711581.6981
35

 683783.7403
36

 649396.9111
37

 599928.1410
38

 594898.2649
39

 577385.1604
40

 573672.7397
41

 561381.6158
42

 546886.3539
43

 455898.6149
44

 429793.0042
45

 389879.8056
46

 380634.4794
47

 351577.9370
48

 335255.1901
49

 332292.5121
50

 310517.0861
51

 304348.4247
52

 300311.4510
53

 279502.4293
54

 271912.1257
55

 262723.3058
56

 253564.3855
57

 249278.5281
58

In [137]:
mae,rmse,auc,precision,recall,f1,ndcg = EvaluationAllUpdated(XPer_GrpMat, rating_matrix_video_4k, k, cutoff)
ResultTrnULS_Grp                      = ["Unified_LS_Grp",mae,rmse,auc,precision,recall,f1,ndcg]
Result_Trn.loc[len(Result_Trn)]       = ["Video Games domain - Run 1",mae,rmse,auc,precision[0][0],precision[0][4],precision[0][9],
                                         precision[0][19],precision[0][29],precision[0][39],recall[0][0],recall[0][4],
                                         recall[0][9],recall[0][19],recall[0][29],recall[0][39],f1[0][0],f1[0][4],f1[0][9],
                                         f1[0][19],f1[0][29],f1[0][39],ndcg[0][0],ndcg[0][4],ndcg[0][9],ndcg[0][19],
                                         ndcg[0][29],ndcg[0][39]]

## GRS for Movies and TV domain

In [465]:
epochs                   = 3
tstPer                   = 30                               # Testing Percentage
NoTstIFromG              = 50
l                        = 5                                # Rating Level

K1                       = 10                               #Number of Clusters in User Space
K2                       = 100                             #Number of Clusters in Item Space

d                        = 100                              # latent space size
minUserPerForSel         = 20                               #Item will be select for test only if minUserPerForSel percentage of users in a group has rated
k                        = 40                               # eval@k: precision@k, recall@k
cutoff                   = 3                                # Relevant > cutoff

lambda1UnifiedLS         = 30
lambda3UnifiedLS         = 1

c2                       = 10**-2
tol                      = 10**-3;
maxiter                  = 500;

In [466]:
n, m = rating_matrix_mt_4k.shape
print(n, m)

6200 13567


In [ ]:
#Spectral Clustering
simU                     = userSim(rating_matrix_mt_4k)
simU                     = (simU + simU.T)/2

UGidx_mt                    = spectralClu(simU,K1).reshape(-1,1)

simI                     = userSim(rating_matrix_mt_4k.T)
simI                     = (simI + simI.T)/2

VGidx_mt                    = spectralClu(simI,K2).reshape(-1,1)

In [439]:
print(UGidx_mt)
print(UGidx_mt.shape)

[[0]
 [0]
 [0]
 ...
 [0]
 [4]
 [4]]
(6200, 1)


In [440]:
print(VGidx_mt)
print(VGidx_mt.shape)

[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]
(13567, 1)


In [441]:
unique, frequency = np.unique(UGidx_mt, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

print()

unique, frequency = np.unique(VGidx_mt, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

Unique Values: [0 1 2 3 4 5 6 7 8 9]
Frequency Values: [4113  208  241  225  261  201  254  183  255  259]

Unique Values: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
Frequency Values: [9290   17  293   42  123  330    2   23  102  245    3  142   80   47
    2   24  321    3   93   10    5    3    3    8    4    6    2    3
   25    4   23    5    2  269    7    4   41   62   54    3   26    3
   14    4    8   10   10    4    2    2   37    4    5    3   12  522
    2    4    3    2    3    3    5    2    2    2    3    3    2    9
   60    3    4    2    6    6  105    4    6    2    2    4   79    3
  321  345    4   31    3    7    2    3    3    6  102    2    2    3
    2    9]


### Recommendation

In [424]:
np.random.seed(0)
vini = np.random.randn((n*d + m*d + K1*d + K2*d), 1)

In [425]:
# Group Recommendation: Unified LS
objGrad             = gradUnifiedLS
v,numiter,ogcalls,J = conjgrad(vini,objGrad,c2,tol,maxiter,l,d,rating_matrix_mt_4k,lambda1UnifiedLS,lambda3UnifiedLS,K1,K2,UGidx_mt,VGidx_mt)
U_mt                  = v[0:n*d].reshape(n,d,order='F')
V_mt                   = v[n*d:n*d+m*d].reshape(m,d,order='F')
UG_mt                  = v[n*d+m*d:n*d+m*d+K1*d].reshape(K1,d,order='F')
X                   = U_mt@V_mt.T
X[X<=1]             = 1
X[X>=5]             = 5
XPerA               = aggregratePrediction(X,UGidx_mt)
XPerAMat_mt            = XPerA[UGidx_mt.reshape(-1,),:]

XPer_Grp                    = UG_mt@V_mt.T
XPer_GrpMat_mt                 = XPer_Grp[UGidx_mt.reshape(-1,),:]
XPer_GrpMat_mt[XPer_GrpMat_mt<=1] = 1
XPer_GrpMat_mt[XPer_GrpMat_mt>=5] = 5


 62461773.3597
1

 43051587.4463
2

 39619824.1056
3

 32534327.8115
4

 31437792.2424
5

 29735002.0243
6

 26289523.1555
7

 26101534.0879
8

 23124813.3933
9

 20168122.1720
10

 19366067.3108
11

 15413182.5111
12

 15087660.9703
13

 14039694.5918
14

 12431485.8599
15

 12035506.5767
16

 10449970.1102
17

 10372680.6545
18

 8875930.2000
19

 8580895.2918
20

 8124322.3361
21

 7832531.6718
22

 6710871.1294
23

 6634814.3354
24

 6226833.8760
25

 5986820.5305
26

 5388383.4402
27

 5323528.7928
28

 4563419.7135
29

 4298053.8292
30

 4034389.5980
31

 3827431.6147
32

 3639449.2788
33

 3284101.0886
34

 3221871.2296
35

 2640412.7471
36

 2618422.2968
37

 2145103.9138
38

 2056312.9402
39

 1790720.3740
40

 1628897.5740
41

 1531300.7015
42

 1472573.1462
43

 1451169.9387
44

 1183919.5249
45

 882772.0757
46

 875035.5745
47

 779911.9582
48

 773017.5146
49

 668168.5376
50

 665849.1613
51

 605118.4036
52

 579640.0099
53

 546943.4485
54

 528511.2944
55

 503627.07

In [322]:
# Result_Trn.drop(index=Result_Trn.index[-1],axis=0,inplace=True)

In [370]:
mae,rmse,auc,precision,recall,f1,ndcg = EvaluationAllUpdated(XPer_GrpMat_mt, rating_matrix_mt_4k, k, cutoff)
ResultTrnULS_Grp                      = ["Unified_LS_Grp_mt",mae,rmse,auc,precision,recall,f1,ndcg]
Result_Trn.loc[len(Result_Trn)]       = ["Movies & TV domain - Run 4",mae,rmse,auc,precision[0][0],precision[0][4],precision[0][9],
                                         precision[0][19],precision[0][29],precision[0][39],recall[0][0],recall[0][4],
                                         recall[0][9],recall[0][19],recall[0][29],recall[0][39],f1[0][0],f1[0][4],f1[0][9],
                                         f1[0][19],f1[0][29],f1[0][39],ndcg[0][0],ndcg[0][4],ndcg[0][9],ndcg[0][19],
                                         ndcg[0][29],ndcg[0][39]]

In [371]:
Result_Trn

,Model,MAE,RMSE,AUC,Precision@1,Precision@5,Precision@10,Precision@20,Precision@30,Precision@40,...,F1@10,F1@20,F1@30,F1@40,NDCG@1,NDCG@5,NDCG@10,NDCG@20,NDCG@30,NDCG@40
0,Video Games domain - Run 1,1.374492,1.590301,0.816004,0.840778,0.820404,0.818276,0.817581,0.817390,0.817337,...,0.832187,0.833731,0.834078,0.834187,0.932460,0.987501,0.993737,0.995641,0.996108,0.996247
1,Movies & TV domain - Run 1,1.256777,1.574522,0.811121,0.905968,0.884030,0.880588,0.879014,0.878419,0.878222,...,0.889444,0.894237,0.895419,0.895918,0.888269,0.971991,0.985653,0.991636,0.993155,0.993871
2,Movies & TV domain - Run 2,1.742414,2.037272,0.739908,0.896774,0.882433,0.879407,0.877905,0.877530,0.877332,...,0.888889,0.893450,0.894752,0.895281,0.891448,0.972241,0.985502,0.991227,0.992850,0.993567
3,Movies & TV domain - Run 3,1.097245,1.267088,0.828392,0.906129,0.884427,0.880899,0.879013,0.878483,0.878290,...,0.890169,0.894356,0.895574,0.896139,0.890308,0.973411,0.986207,0.992091,0.993767,0.994540


## Creating X, Y matrices for mapping

In [372]:
#Adding individual latent vectors
for user_id in train:
    source_ind = np.where(new_user_ids_video_4k == user_id)
    target_ind = np.where(new_user_ids_mt_4k == user_id)
    X_train = np.concatenate((X_train, U_video[source_ind]), axis = 0)
    Y_train = np.concatenate((Y_train, U_mt[target_ind]), axis = 0)
    
for user_id in test:
    source_ind = np.where(new_user_ids_video_4k == user_id)
    target_ind = np.where(new_user_ids_mt_4k == user_id)
    X_test = np.concatenate((X_test, U_video[source_ind]), axis = 0)  

    
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(3200, 100)
(3200, 100)
(800, 100)
(800, 13567)


In [373]:
groupsToMembers_video = []
groupsToMembers_mt = []
for i in range(K1):
    groupsToMembers_video.append([])
    groupsToMembers_mt.append([])

for i in range(UGidx_video.shape[0]):
    groupsToMembers_video[int(UGidx_video[i][0])].append(new_user_ids_video_4k[i])
    
for i in range(UGidx_mt.shape[0]):
    groupsToMembers_mt[int(UGidx_mt[i][0])].append(new_user_ids_mt_4k[i])
    
print(len(groupsToMembers_video))
print(len(groupsToMembers_mt))

10
10


In [374]:
for i in range(len(groupsToMembers_video)):
    print(len(groupsToMembers_video[i]))

259
6009
320
316
290
313
329
371
424
369


In [375]:
for i in range(len(groupsToMembers_mt)):
    print(len(groupsToMembers_mt[i]))

213
4130
241
232
231
258
260
220
238
177


In [376]:
print(groupsToMembers_mt[5])

['A19YSCD4FCGE4Y', 'A31255YKR4NJS1', 'A3JZXERHBSK7RL', 'A1DEXIE3Y46XU1', 'AN7NMF5BNVTHG', 'A26MZUISRSKSRL', 'AAUUJQIWFWXOB', 'A2665IPGY4RFIR', 'A33W2MIW13OY2C', 'A3VYTZVNGCRJY', 'A3RAJ9U9QKOAB0', 'A167QI6IUH1RJJ', 'A65IRI986VOD4', 'A2ZFHSRRKZ4YQ7', 'A3OCKYJNMQMKBL', 'A31W0JBM36Y9T1', 'ADTM58S9N4N99', 'AOFC7AHH6X5K', 'A1YI5FROZCH0JX', 'ANEMUCMAODB61', 'A399ONPL55JCH6', 'A3T5GPJL56AI70', 'A1E5FVOVL1R7PU', 'A184ABO0ATT20T', 'AD5JU20ROYBJ4', 'A3RXOEOGGR9IU3', 'AGCPRHBG7KIEJ', 'A3FT7EY5ULAVWM', 'A33XT7W4Q7AZ0I', 'AXG8UKTE4W8ZO', 'A2AZS3A7WDNGMT', 'A1MB35OXDXCMNE', 'APJB6R6H5V515', 'AYWSIKLQSHYNX', 'A3QSGYDDEG858B', 'A2PVX0LIQMXCLM', 'A2MVVQZDMSH7UN', 'A1YW5MUPK5JKMT', 'A8JBTRSSAHACP', 'A2J7UNGL0LUNSO', 'A3G5EJ63OLB6V6', 'A10R72UOSE0P86', 'A2D937VH6V7TII', 'ASSIRDSY1Q1F7', 'A11FKTWOGUTA0M', 'A2DMRH25Z5CKYB', 'A2L9XEFJS80RGD', 'A3PMS2KPPXQ531', 'A2N146LF8EWJAC', 'A15HRQ3HS9N2FO', 'A42S5Q0WN2HIM', 'A3O2WXBF5ONSLO', 'A1PMV3RCO2MOBT', 'A3RV6KPBD1OZWP', 'A1TNUH871PF4KI', 'A1JIDY3K8WAKIS', 'A26PD7

In [377]:
counter = 0
for i in range(len(groupsToMembers_video)):
    currGroup_video = groupsToMembers_video[i]
    for j in range(len(groupsToMembers_mt)):
        currGroup_mt = groupsToMembers_mt[j]
        if len(set(currGroup_video).intersection(set(currGroup_mt))) > 0:
            counter = counter + 1
            X_train = np.row_stack((X_train, UG_video[i]))
            Y_train = np.row_stack((Y_train, UG_mt[j]))
            
print(X_train.shape)
print(Y_train.shape)
print(counter)

(3298, 100)
(3298, 100)
98


## MLP Mapping

In [378]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.optim as optim

In [379]:
class GetDataset(torch.utils.data.Dataset):

  def __init__(self, X, y, scale_data=True):
    if not torch.is_tensor(X) and not torch.is_tensor(y):
      self.X = torch.from_numpy(X)
      self.y = torch.from_numpy(y)

  def __len__(self):
      return len(self.X)

  def __getitem__(self, i):
      return self.X[i], self.y[i]

In [380]:
class MLP(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(100, 64),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.ReLU(),
      nn.Linear(32, 100)
    )


  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)

In [381]:
# Set fixed random number seed
torch.manual_seed(43)

In [382]:
dataset = GetDataset(X_train, Y_train)
trainloader = torch.utils.data.DataLoader(dataset, batch_size=X_train.shape[0], shuffle=False, num_workers=0)

In [178]:
space = [
    Real(1e-3, 5e-2, name='lr'),
    Real(1e-5, 1e-2, name='decay')
]

In [179]:
def trainMLPAndGetMAE(hyperparameters):
    # Extract hyperparameters
    learning_rate = hyperparameters[0]
    wt_decay = hyperparameters[1]
    
    mlp = MLP()
  
    # Define the loss function and optimizer
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate, weight_decay=wt_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=1, verbose=True)
    
    counter = 0
    # Run the training loop
    while optimizer.param_groups[-1]['lr'] > 1e-4: # 5 epochs at maximum

        # Print epoch
    #     print(f'Starting epoch {epoch+1}')

        # Set current loss value
        current_loss = 0.0
        losses = []
        # Iterate over the DataLoader for training data
        for i, data in enumerate(trainloader, 0):

          # Get and prepare inputs
    #         if counter > 2:
    #             break
    #         print(counter)
            inputs, targets = data
            inputs, targets = inputs.float(), targets.float()
            targets = targets.reshape((targets.shape[0], 100))

    #         print("Inputs shape: ", inputs.shape)
    #         print("Target shape: ", targets.shape)
    #         counter += 1

          # Zero the gradients
            optimizer.zero_grad()

          # Perform forward pass
            outputs = mlp(inputs)
    #         print("Output shape: ", outputs.shape)

#             outputs_np = outputs.detach().numpy()
#             targets_np = targets.detach().numpy()
            t_V_mt = torch.from_numpy(V_mt.T).float()

#             t_vec = targets_np@V_mt.T
#             o_vec = outputs_np@V_mt.T
            t_vec = torch.matmul(targets, t_V_mt)
            o_vec = torch.matmul(outputs, t_V_mt)
    #         print("t_vec.shape: ", t_vec.shape)
    #         print("o_vec.shape: ", o_vec.shape)
            indicator_matrix = (t_vec >= 0.7)
    #         print("indicator matrix: ", indicator_matrix)
    #         print(np.all(indicator_matrix == True))
#             t_vec_interacted = torch.from_numpy(t_vec*indicator_matrix).requires_grad_() 
#             o_vec_interacted = torch.from_numpy(o_vec*indicator_matrix).requires_grad_() 
            t_vec_interacted = torch.mul(t_vec, indicator_matrix)
            o_vec_interacted = torch.mul(o_vec, indicator_matrix)
#             t = torch.from_numpy(a)

          # Compute loss
            loss = loss_function(o_vec_interacted, t_vec_interacted)
    #         print("Loss: ", loss.item())
            print("Epoch : " + str(counter + 1) + " | Error : " + str(loss.item()) + " | LR : " + str(optimizer.param_groups[-1]['lr']))
            losses.append(loss.item())
          # Perform backward pass
            loss.backward()

          # Perform optimization
            optimizer.step()


        mean_loss = sum(losses) / len(losses)
        scheduler.step(mean_loss)
#         print(f"Loss at epoch {counter + 1} = {mean_loss}")
        if optimizer.param_groups[-1]['lr'] <= 1e-4:
            break;
        counter += 1

    # Process is complete.
    print('Training process has finished.')
    
    pred_X_train = mlp(torch.from_numpy(np.float32(X_train))).detach().numpy()
    print("pred_X_train: ", pred_X_train.shape)
    par1 = pred_X_train@V_mt.T
    par2 = Y_train@V_mt.T
    mae = MAE(par1, par2)
    print("MAE: ", mae)
    print()
    print()
    return mae

In [180]:
def objective(hyperparameters):
    print(hyperparameters)
    return trainMLPAndGetMAE(hyperparameters)

In [181]:
# Perform Bayesian optimization
result = gp_minimize(objective, space, n_calls=20)

[0.026383014169248452, 0.003297965355660687]
Epoch : 1 | Error : 9.919313430786133 | LR : 0.026383014169248452
Epoch : 2 | Error : 1.261682391166687 | LR : 0.026383014169248452
Epoch : 3 | Error : 56.91688537597656 | LR : 0.026383014169248452
Epoch : 4 | Error : 2.5525522232055664 | LR : 0.026383014169248452
Epoch 00004: reducing learning rate of group 0 to 2.6383e-03.
Epoch : 5 | Error : 3.055696725845337 | LR : 0.0026383014169248454
Epoch : 6 | Error : 3.4064033031463623 | LR : 0.0026383014169248454
Epoch 00006: reducing learning rate of group 0 to 2.6383e-04.
Epoch : 7 | Error : 3.6609067916870117 | LR : 0.00026383014169248457
Epoch : 8 | Error : 3.6787192821502686 | LR : 0.00026383014169248457
Epoch 00008: reducing learning rate of group 0 to 2.6383e-05.
Training process has finished.
pred_X_train:  (3299, 100)
MAE:  1.6555954229623289


[0.00460732644632037, 0.0017364273656163428]
Epoch : 1 | Error : 9.274444580078125 | LR : 0.00460732644632037
Epoch : 2 | Error : 8.22262382507324

Epoch : 90 | Error : 0.8232207298278809 | LR : 0.00017343925332154246
Epoch : 91 | Error : 0.8231067657470703 | LR : 0.00017343925332154246
Epoch : 92 | Error : 0.8229925036430359 | LR : 0.00017343925332154246
Epoch : 93 | Error : 0.8228774666786194 | LR : 0.00017343925332154246
Epoch : 94 | Error : 0.8227623701095581 | LR : 0.00017343925332154246
Epoch : 95 | Error : 0.8226468563079834 | LR : 0.00017343925332154246
Epoch : 96 | Error : 0.8225315809249878 | LR : 0.00017343925332154246
Epoch : 97 | Error : 0.8224157691001892 | LR : 0.00017343925332154246
Epoch : 98 | Error : 0.8222996592521667 | LR : 0.00017343925332154246
Epoch : 99 | Error : 0.8221831321716309 | LR : 0.00017343925332154246
Epoch : 100 | Error : 0.8220664858818054 | LR : 0.00017343925332154246
Epoch : 101 | Error : 0.8219496011734009 | LR : 0.00017343925332154246
Epoch : 102 | Error : 0.821832537651062 | LR : 0.00017343925332154246
Epoch : 103 | Error : 0.8217151761054993 | LR : 0.00017343925332154246
Epoch : 104 | Err

Epoch : 206 | Error : 0.8092967867851257 | LR : 0.00017343925332154246
Epoch : 207 | Error : 0.8091750741004944 | LR : 0.00017343925332154246
Epoch : 208 | Error : 0.8090534806251526 | LR : 0.00017343925332154246
Epoch : 209 | Error : 0.8089317083358765 | LR : 0.00017343925332154246
Epoch : 210 | Error : 0.8088101744651794 | LR : 0.00017343925332154246
Epoch : 211 | Error : 0.8086886405944824 | LR : 0.00017343925332154246
Epoch : 212 | Error : 0.8085669875144958 | LR : 0.00017343925332154246
Epoch : 213 | Error : 0.8084455728530884 | LR : 0.00017343925332154246
Epoch : 214 | Error : 0.8083240389823914 | LR : 0.00017343925332154246
Epoch : 215 | Error : 0.8082025647163391 | LR : 0.00017343925332154246
Epoch : 216 | Error : 0.8080812096595764 | LR : 0.00017343925332154246
Epoch : 217 | Error : 0.8079598546028137 | LR : 0.00017343925332154246
Epoch : 218 | Error : 0.8078384399414062 | LR : 0.00017343925332154246
Epoch : 219 | Error : 0.8077172636985779 | LR : 0.00017343925332154246
Epoch 

Epoch : 322 | Error : 0.7956596612930298 | LR : 0.00017343925332154246
Epoch : 323 | Error : 0.795545220375061 | LR : 0.00017343925332154246
Epoch : 324 | Error : 0.7954300045967102 | LR : 0.00017343925332154246
Epoch : 325 | Error : 0.7953142523765564 | LR : 0.00017343925332154246
Epoch : 326 | Error : 0.7951982617378235 | LR : 0.00017343925332154246
Epoch : 327 | Error : 0.7950814366340637 | LR : 0.00017343925332154246
Epoch : 328 | Error : 0.7949641346931458 | LR : 0.00017343925332154246
Epoch : 329 | Error : 0.7948458194732666 | LR : 0.00017343925332154246
Epoch : 330 | Error : 0.7947270274162292 | LR : 0.00017343925332154246
Epoch : 331 | Error : 0.7946069836616516 | LR : 0.00017343925332154246
Epoch : 332 | Error : 0.7944850921630859 | LR : 0.00017343925332154246
Epoch : 333 | Error : 0.794361412525177 | LR : 0.00017343925332154246
Epoch : 334 | Error : 0.7942292094230652 | LR : 0.00017343925332154246
Epoch : 335 | Error : 0.7940893769264221 | LR : 0.00017343925332154246
Epoch : 

Epoch : 438 | Error : 0.7783693075180054 | LR : 0.00017343925332154246
Epoch : 439 | Error : 0.7782195210456848 | LR : 0.00017343925332154246
Epoch : 440 | Error : 0.7780706286430359 | LR : 0.00017343925332154246
Epoch : 441 | Error : 0.777921736240387 | LR : 0.00017343925332154246
Epoch : 442 | Error : 0.777773380279541 | LR : 0.00017343925332154246
Epoch : 443 | Error : 0.7776254415512085 | LR : 0.00017343925332154246
Epoch : 444 | Error : 0.7774775624275208 | LR : 0.00017343925332154246
Epoch : 445 | Error : 0.7773304581642151 | LR : 0.00017343925332154246
Epoch : 446 | Error : 0.7771834135055542 | LR : 0.00017343925332154246
Epoch : 447 | Error : 0.7770361304283142 | LR : 0.00017343925332154246
Epoch : 448 | Error : 0.7768898606300354 | LR : 0.00017343925332154246
Epoch : 449 | Error : 0.7767435312271118 | LR : 0.00017343925332154246
Epoch : 450 | Error : 0.7765979766845703 | LR : 0.00017343925332154246
Epoch : 451 | Error : 0.7764525413513184 | LR : 0.00017343925332154246
Epoch : 

...

Epoch : 169 | Error : 0.7771558165550232 | LR : 0.00047918645952238286
Epoch : 170 | Error : 0.7771139740943909 | LR : 0.00047918645952238286
Epoch : 171 | Error : 0.7770723104476929 | LR : 0.00047918645952238286
Epoch : 172 | Error : 0.7770307660102844 | LR : 0.00047918645952238286
Epoch : 173 | Error : 0.7769893407821655 | LR : 0.00047918645952238286
Epoch : 174 | Error : 0.776948094367981 | LR : 0.00047918645952238286
Epoch : 175 | Error : 0.7769068479537964 | LR : 0.00047918645952238286
Epoch : 176 | Error : 0.7768657207489014 | LR : 0.00047918645952238286
Epoch : 177 | Error : 0.776825487613678 | LR : 0.00047918645952238286
Epoch : 178 | Error : 0.7767854332923889 | LR : 0.00047918645952238286
Epoch : 179 | Error : 0.7767456769943237 | LR : 0.00047918645952238286
Epoch : 180 | Error : 0.7767057418823242 | LR : 0.00047918645952238286
Epoch : 181 | Error : 0.776665985584259 | LR : 0.00047918645952238286
Epoch : 182 | Error : 0.7766260504722595 | LR : 0.00047918645952238286
Epoch : 1

Epoch : 15 | Error : 0.8341874480247498 | LR : 0.0019097208925659406
Epoch 00015: reducing learning rate of group 0 to 1.9097e-04.
Epoch : 16 | Error : 0.8377742767333984 | LR : 0.00019097208925659406
Epoch : 17 | Error : 0.8378689885139465 | LR : 0.00019097208925659406
Epoch 00017: reducing learning rate of group 0 to 1.9097e-05.
Training process has finished.
pred_X_train:  (3299, 100)
MAE:  0.7414976412270434


[0.04569539981827977, 0.0005229067975243095]
Epoch : 1 | Error : 10.874434471130371 | LR : 0.04569539981827977
Epoch : 2 | Error : 4.31588077545166 | LR : 0.04569539981827977
Epoch : 3 | Error : 5.390632152557373 | LR : 0.04569539981827977
Epoch : 4 | Error : 4.845002174377441 | LR : 0.04569539981827977
Epoch 00004: reducing learning rate of group 0 to 4.5695e-03.
Epoch : 5 | Error : 3.4314334392547607 | LR : 0.004569539981827977
Epoch : 6 | Error : 3.244518756866455 | LR : 0.004569539981827977
Epoch : 7 | Error : 3.0197346210479736 | LR : 0.004569539981827977
Epoch : 8 | Err

Epoch : 27 | Error : 1.38580322265625 | LR : 0.0003426210894339747
Epoch : 28 | Error : 1.3693851232528687 | LR : 0.0003426210894339747
Epoch : 29 | Error : 1.3531954288482666 | LR : 0.0003426210894339747
Epoch : 30 | Error : 1.337207555770874 | LR : 0.0003426210894339747
Epoch : 31 | Error : 1.3213969469070435 | LR : 0.0003426210894339747
Epoch : 32 | Error : 1.3057401180267334 | LR : 0.0003426210894339747
Epoch : 33 | Error : 1.290221929550171 | LR : 0.0003426210894339747
Epoch : 34 | Error : 1.274827241897583 | LR : 0.0003426210894339747
Epoch : 35 | Error : 1.2595529556274414 | LR : 0.0003426210894339747
Epoch : 36 | Error : 1.2443900108337402 | LR : 0.0003426210894339747
Epoch : 37 | Error : 1.2293442487716675 | LR : 0.0003426210894339747
Epoch : 38 | Error : 1.2144187688827515 | LR : 0.0003426210894339747
Epoch : 39 | Error : 1.199623465538025 | LR : 0.0003426210894339747
Epoch : 40 | Error : 1.1849607229232788 | LR : 0.0003426210894339747
Epoch : 41 | Error : 1.1704362630844116 

Epoch : 146 | Error : 0.7827379107475281 | LR : 0.0003426210894339747
Epoch : 147 | Error : 0.7826632857322693 | LR : 0.0003426210894339747
Epoch : 148 | Error : 0.7825884819030762 | LR : 0.0003426210894339747
Epoch : 149 | Error : 0.7825136780738831 | LR : 0.0003426210894339747
Epoch : 150 | Error : 0.7824392318725586 | LR : 0.0003426210894339747
Epoch : 151 | Error : 0.7823677062988281 | LR : 0.0003426210894339747
Epoch : 152 | Error : 0.7822965979576111 | LR : 0.0003426210894339747
Epoch : 153 | Error : 0.7822254300117493 | LR : 0.0003426210894339747
Epoch : 154 | Error : 0.7821559906005859 | LR : 0.0003426210894339747
Epoch : 155 | Error : 0.7820860147476196 | LR : 0.0003426210894339747
Epoch : 156 | Error : 0.7820162177085876 | LR : 0.0003426210894339747
Epoch : 157 | Error : 0.7819488644599915 | LR : 0.0003426210894339747
Epoch : 158 | Error : 0.7818835973739624 | LR : 0.0003426210894339747
Epoch : 159 | Error : 0.7818195819854736 | LR : 0.0003426210894339747
Epoch : 160 | Error 

Epoch : 264 | Error : 0.7762121558189392 | LR : 0.0003426210894339747
Epoch : 265 | Error : 0.7761648893356323 | LR : 0.0003426210894339747
Epoch : 266 | Error : 0.7761178612709045 | LR : 0.0003426210894339747
Epoch : 267 | Error : 0.7760709524154663 | LR : 0.0003426210894339747
Epoch : 268 | Error : 0.7760242223739624 | LR : 0.0003426210894339747
Epoch : 269 | Error : 0.7759774327278137 | LR : 0.0003426210894339747
Epoch : 270 | Error : 0.7759308815002441 | LR : 0.0003426210894339747
Epoch : 271 | Error : 0.7758844494819641 | LR : 0.0003426210894339747
Epoch : 272 | Error : 0.7758380174636841 | LR : 0.0003426210894339747
Epoch : 273 | Error : 0.7757919430732727 | LR : 0.0003426210894339747
Epoch : 274 | Error : 0.7757458090782166 | LR : 0.0003426210894339747
Epoch : 275 | Error : 0.7756998538970947 | LR : 0.0003426210894339747
Epoch : 276 | Error : 0.7756539583206177 | LR : 0.0003426210894339747
Epoch : 277 | Error : 0.7756083607673645 | LR : 0.0003426210894339747
Epoch : 278 | Error 

In [182]:
# Get the best hyperparameters and performance
# best_hyperparameters = {
#     'K1' = result.x[0]
#     'K2' = result.x[1]
#     'lambda1UnifiedLS' = result.x[2]
#     'lambda3UnifiedLS' = result.x[3]
# }
best_performance = result.fun
print("Best MAE: {:.4f}".format(result.fun))
print("Best Hyperparameters: {}".format(dict(zip(['lr', 'decay'], result.x))))

Best MAE: 0.6488
Best Hyperparameters: {'lr': 0.045713698797232624, 'decay': 1e-05}


In [183]:
# Print the MAE values and hyperparameters for each call
for i, val in enumerate(result.func_vals):
    print("Call {}: MAE={:.4f}, Hyperparameters={}".format(i, val, result.x_iters[i]))

Call 0: MAE=1.6556, Hyperparameters=[0.026383014169248452, 0.003297965355660687]
Call 1: MAE=1.0354, Hyperparameters=[0.00460732644632037, 0.0017364273656163428]
Call 2: MAE=0.7481, Hyperparameters=[0.017343925332154245, 0.00720494508745866]
Call 3: MAE=1.0502, Hyperparameters=[0.0017944161032790872, 0.005955732633154312]
Call 4: MAE=1.9121, Hyperparameters=[0.033392859956814444, 0.008404280953008828]
Call 5: MAE=0.9388, Hyperparameters=[0.04726872639179779, 0.0034334733649864116]
Call 6: MAE=0.7470, Hyperparameters=[0.044586821652320925, 0.0016282676078397769]
Call 7: MAE=2.0971, Hyperparameters=[0.04399976755428895, 0.005936272133057427]
Call 8: MAE=0.7404, Hyperparameters=[0.04795586289467641, 0.006371906606308638]
Call 9: MAE=0.9052, Hyperparameters=[0.022201167872106995, 0.003172951193128692]
Call 10: MAE=0.7601, Hyperparameters=[0.04791864595223828, 0.006168011910040755]
Call 11: MAE=0.6488, Hyperparameters=[0.045713698797232624, 1e-05]
Call 12: MAE=0.8067, Hyperparameters=[0.044

In [184]:
# Print the values of the optimized hyperparameters
print('Optimized Hyperparameters:')
for hyperparameter, value in zip(space, result.x):
    print('{}: {}'.format(hyperparameter.name, value))

Optimized Hyperparameters:
lr: 0.045713698797232624
decay: 1e-05


## MLP with tuned hyperparameters

In [383]:
torch.manual_seed(45)

In [384]:
mlp = MLP()
  
# Define the loss function and optimizer
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.045713698797232624, weight_decay=1e-05)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=1, verbose=True)

In [385]:
counter = 0
# Run the training loop
while optimizer.param_groups[-1]['lr'] > 1e-4: # 5 epochs at maximum
    
    # Print epoch
#     print(f'Starting epoch {epoch+1}')
    
    # Set current loss value
    current_loss = 0.0
    losses = []
    # Iterate over the DataLoader for training data
    for i, data in enumerate(trainloader, 0):
      
      # Get and prepare inputs
#         if counter > 2:
#             break
#         print(counter)
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 100))
        
#         print("Inputs shape: ", inputs.shape)
#         print("Target shape: ", targets.shape)
#         counter += 1
      
      # Zero the gradients
        optimizer.zero_grad()
      
      # Perform forward pass
        outputs = mlp(inputs)
#         print("Output shape: ", outputs.shape)
        
#         outputs_np = outputs.detach().numpy()
#         targets_np = targets.detach().numpy()
        t_V_mt = torch.from_numpy(V_mt.T).float()
    
#         t_vec = targets_np@V_mt.T
#         o_vec = outputs_np@V_mt.T
#         print("t_vec.shape: ", t_vec.shape)
#         print("o_vec.shape: ", o_vec.shape)
        t_vec = torch.matmul(targets, t_V_mt)
        o_vec = torch.matmul(outputs, t_V_mt)
        indicator_matrix = (t_vec >= 0.7)
#         print("indicator matrix: ", indicator_matrix)
#         print(np.all(indicator_matrix == True))
#         t_vec_interacted = t_vec*indicator_matrix
#         o_vec_interacted = o_vec*indicator_matrix
        t_vec_interacted = torch.mul(t_vec, indicator_matrix)
        o_vec_interacted = torch.mul(o_vec, indicator_matrix)
#             t = torch.from_numpy(a)

        # Compute loss
        loss = loss_function(o_vec_interacted, t_vec_interacted)
        
#         print("Loss: ", loss.item())
        print("Epoch : " + str(counter + 1) + " | Error : " + str(loss.item()) + " | LR : " + str(optimizer.param_groups[-1]['lr']))
        losses.append(loss.item())
      # Perform backward pass
        loss.backward()
      
      # Perform optimization
        optimizer.step()
        
        
    mean_loss = sum(losses) / len(losses)
    scheduler.step(mean_loss)
    print(f"Loss at epoch {counter + 1} = {mean_loss}")
    if optimizer.param_groups[-1]['lr'] <= 1e-4:
        break;
    counter += 1

# Process is complete.
print('Training process has finished.')

Epoch : 1 | Error : 11.813236236572266 | LR : 0.045713698797232624
Loss at epoch 1 = 11.813236236572266
Epoch : 2 | Error : 9.674954414367676 | LR : 0.045713698797232624
Loss at epoch 2 = 9.674954414367676
Epoch : 3 | Error : 7.039244174957275 | LR : 0.045713698797232624
Loss at epoch 3 = 7.039244174957275
Epoch : 4 | Error : 6.416309833526611 | LR : 0.045713698797232624
Loss at epoch 4 = 6.416309833526611
Epoch : 5 | Error : 4.903297424316406 | LR : 0.045713698797232624
Loss at epoch 5 = 4.903297424316406
Epoch : 6 | Error : 3.0134353637695312 | LR : 0.045713698797232624
Loss at epoch 6 = 3.0134353637695312
Epoch : 7 | Error : 1.2746167182922363 | LR : 0.045713698797232624
Loss at epoch 7 = 1.2746167182922363
Epoch : 8 | Error : 0.8177812695503235 | LR : 0.045713698797232624
Loss at epoch 8 = 0.8177812695503235
Epoch : 9 | Error : 0.8305140733718872 | LR : 0.045713698797232624
Loss at epoch 9 = 0.8305140733718872
Epoch : 10 | Error : 0.7934491038322449 | LR : 0.045713698797232624
Loss

Epoch : 78 | Error : 0.10232016444206238 | LR : 0.045713698797232624
Loss at epoch 78 = 0.10232016444206238
Epoch : 79 | Error : 0.10221836715936661 | LR : 0.045713698797232624
Loss at epoch 79 = 0.10221836715936661
Epoch : 80 | Error : 0.1021813154220581 | LR : 0.045713698797232624
Loss at epoch 80 = 0.1021813154220581
Epoch : 81 | Error : 0.10218191891908646 | LR : 0.045713698797232624
Loss at epoch 81 = 0.10218191891908646
Epoch : 82 | Error : 0.10218145698308945 | LR : 0.045713698797232624
Epoch 00082: reducing learning rate of group 0 to 4.5714e-03.
Loss at epoch 82 = 0.10218145698308945
Epoch : 83 | Error : 0.10233023762702942 | LR : 0.004571369879723263
Loss at epoch 83 = 0.10233023762702942
Epoch : 84 | Error : 0.10210075974464417 | LR : 0.004571369879723263
Loss at epoch 84 = 0.10210075974464417
Epoch : 85 | Error : 0.10178927332162857 | LR : 0.004571369879723263
Loss at epoch 85 = 0.10178927332162857
Epoch : 86 | Error : 0.10173852741718292 | LR : 0.004571369879723263
Loss at

In [386]:
pred_X_train = mlp(torch.from_numpy(np.float32(X_train))).detach().numpy()
print(pred_X_train.shape)
cutoff = 3
k = 40
par1 = pred_X_train@V_mt.T
par2 = Y_train@V_mt.T
# mae,rmse = EvaluationAllUpdated_N(par1, par2, k, cutoff)
# print("Training Metrics")
# print(mae, rmse)

(3298, 100)


In [387]:
mae,rmse = EvaluationAllUpdated_N(par1, par2, k, cutoff)
Training_Metrics                = ["CDGRS_Train",mae,rmse]
Result_Trn_CDR.loc[len(Result_Trn_CDR)]       = ["Train-3.3",mae,rmse]

In [388]:
pred_X_test = mlp(torch.from_numpy(np.float32(X_test))).detach().numpy()
print(pred_X_test.shape)
cutoff = 3
k = 40
par1 = pred_X_test@V_mt.T
par1[par1 > 5] = 5
par2 = Y_test
# mae,rmse = EvaluationAllUpdated_N(par1, par2, k, cutoff)
# print("Test Metrics")
# print(mae, rmse)

(800, 100)


In [389]:
mae,rmse = EvaluationAllUpdated_N(par1, par2, k, cutoff)
Testing_Metrics                = ["CDGRS_Test",mae,rmse]
Result_Tst_CDR.loc[len(Result_Tst_CDR)]       = ["Test-3.3",mae,rmse]

In [390]:
Result_Trn_CDR

,Model,MAE,RMSE
0,Train-3,0.714461,0.913378
1,Train-3.1,0.652411,0.933357
2,Train-3.2,0.243811,0.320318


In [391]:
Result_Trn_CDR.mean()

C:\Users\NIT\AppData\Local\Temp\ipykernel_22436\3129751611.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  Result_Trn_CDR.mean()


MAE     0.536894
RMSE    0.722351
dtype: float64

In [392]:
Result_Tst_CDR

,Model,MAE,RMSE
0,Test-3,1.496920,1.730555
1,Test-3.1,1.877714,2.115850
2,Test-3.2,1.262800,1.406024


In [393]:
Result_Tst_CDR.mean()

C:\Users\NIT\AppData\Local\Temp\ipykernel_22436\661790523.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  Result_Tst_CDR.mean()


MAE     1.545811
RMSE    1.750810
dtype: float64

In [321]:
# Result_Trn_CDR.drop(index=Result_Trn_CDR.index[-1],axis=0,inplace=True)
# Result_Tst_CDR.drop(index=Result_Tst_CDR.index[-1],axis=0,inplace=True)